In [1]:
import numpy as np
import pandas as pd
from fractions import Fraction
import xlsxwriter
import decimal
from decimal import *
import re
from modsigfig import round #does this need to be done 
pd.set_option('display.max_rows', 500)
#allowed: p to s, d, p. s to p. d to s, d.
# n anything below energy
# j = 1/2 to 1/2, 3/2. j = 3/2 to 3/2, 1/2, 5/2. j = 5/2 to 5/2, 3/2, 7/2. 

In [2]:
round(1359.2016, 0.0006, format = 'Drake', err_dig = 1)

C:\Users\dmgame\Documents\SafronovaResearch\LifetimesWebsite\modsigfig.py:572: UserWarning: warning: 2 significant figures requested from number with only 1 significant figures
  warn("warning: %d significant figures requested from number with only %d significant figures" % (given['sigfigs'], len(num.map)))


'1359.2016(6)'

In [3]:
element = input('Element name in files:')
skiprows_data = 0 #for database
skiprows_mats = 1 #for datapol
dataname = "Data\%s\database%s.txt" % (element,element) #used in getting (#) format of erros
matname = "Data\%s\datapol%s.txt" % (element, element) #the rest of the data
checkratesname = "Data\\%s\\rates1.txt" % element
checklifesname = "Data\\%s\\rates2.txt" % element

Element name in files:Cs


In [4]:
# database = pd.read_csv(dataname, sep = '\s+', engine = 'python', header = None, skiprows = skiprows_data)
# real_mats = []
# for d, i in enumerate(database[5]):
#     if i == None:
#         real_mats.append(database[4][d])
#     else:
#         real_mats.append(i)
# database[5] = real_mats
# database.drop(columns = [0,1,2,3,4], inplace = True)
# database
# columns = ['matrix']
# database.columns = columns
# database[['matrix', 'unc']] = database['matrix'].str.split('(', expand = True)
# database[['unc', 'trash']] = database['unc'].str.split(')', expand = True)
# database.drop('trash',axis = 1, inplace = True)

# #have to read in as str or else pandas does weird rounding. Convert numbers back to float after
# ele = pd.read_csv(matname, sep = '\s+', engine = 'python', header = None, skiprows = skiprows_mats, dtype = 'str')
# ele[2] = ele[2].astype('float64')
# ele[3] = ele[3].astype('float64')
# cutoff_index = ele[ele[1].isnull()].index[0]
# mats = ele[:cutoff_index]
# energies = ele[cutoff_index+1:]
# energies[1] = energies[1].astype('float64')
# energies[2] = energies[2].astype('float64')

# columns = ['start', 'end','matrix', 'unc']
# mats.columns = columns
# mats['old_unc'] = database['unc']
# columns2 = ['level', 'ene', 'unc', 'modif' ]
# energies.columns = columns2
# energies.reset_index(inplace = True, drop = True)

# # for index, row in mats.iterrows():
# #     if 'f' in row.end:
# #         mats.drop(index, inplace=True)
# #     elif 'f' in row.start:
# #         mats.drop(index, inplace=True)
# mats.reset_index(inplace = True,drop = True)

In [5]:
database = pd.read_csv(dataname, sep = '\s+', engine = 'python', header = None, skiprows = skiprows_data)
real_mats = []
for d, i in enumerate(database[5]):
    if i == None:
        real_mats.append(database[4][d])
    else:
        real_mats.append(i)
database[5] = real_mats
database.drop(columns = [0,1,2,3,4], inplace = True)
database
columns = ['matrix']
database.columns = columns
database[['matrix', 'unc']] = database['matrix'].str.split('(', expand = True)
database[['unc', 'trash']] = database['unc'].str.split(')', expand = True)
database.drop('trash',axis = 1, inplace = True)

#have to read in as str or else pandas does weird rounding. Convert numbers back to float after
ele = pd.read_csv(matname, sep = '\s+', engine = 'python', header = None, skiprows = skiprows_mats, dtype = 'str')
cutoff_index = ele[ele[1].isnull()].index[0]
mats = ele[:cutoff_index]
energies = ele[cutoff_index+1:]

columns = ['start', 'end','matrix', 'unc']
mats.columns = columns
mats['old_unc'] = database['unc']
columns2 = ['level', 'ene', 'unc', 'modif' ]
energies.columns = columns2
energies.reset_index(inplace = True, drop = True)

# for index, row in mats.iterrows():
#     if 'f' in row.end:
#         mats.drop(index, inplace=True)
#     elif 'f' in row.start:
#         mats.drop(index, inplace=True)
mats.reset_index(inplace = True,drop = True)

<ipython-input-5-316b4437bd5d>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mats['old_unc'] = database['unc']


In [6]:
#Processing of mats file off of end state, since ground state will not decay
#creates dataframe of all possible beginning states
def split(word): 
    return list(word) 

all_names = []
#splits the original n+l number into components
for i in mats['end']:
    nm = split(i)
    name = [string for string in nm if string != " "]
    all_names.append(name)

# fg_hold = []
# for i in range(len(all_names)):
#     if 'f' in all_names[i]: 
#         #print(all_names[i])
#         fg_hold.append(i)
   
# all_names = np.delete(all_names, fg_hold).tolist()
#changes s, p, d into 0, 1, 2
for i in all_names:
    if 's' in i:
        i[i.index('s')] = 0
    elif 'p' in i:
        i[i.index('p')] = 1
    elif 'd' in i:
        i[i.index('d')] = 2
    elif 'f' in i:
        i[i.index('f')] = 3
    elif 'g' in i:
        i[i.index('g')] = 4
    
#changes n element into integer, whether its 1 or 2 digits
for i in all_names:
    if isinstance(i[1], str): #means it hasnt been changed to 0,1,2 which means its a 2 digit n value. i.e. 12
        i[0] = int(i[0] + i[1])
        del i[1]
    else:
        i[0] = int(i[0])

#converts last 3 numbers into 1 j value
for i in all_names:
    i[2] = int(i[2]) / int(i[4])
    del i[4]
    del i[3]

#adds back in matrix elements and uncertainty, starts column titles
for count, i in enumerate(all_names):
    i.append(mats.matrix[count])
    i.append(mats.unc[count])
columns = ['n','l','j','m','unc']
ends = pd.DataFrame(all_names, columns=columns)

In [7]:
#Processing of mats file off of start state,
#creates dataframe of some decay states. Not all of them, as n is not included

all_names = []
#splits the original n+l number into components
for i in mats['start']:
    nm = split(i)
    name = [string for string in nm if string != " "]
    all_names.append(name)
 

#changes s, p, d into 0, 1, 2
for i in all_names:
    if 's' in i:
        i[i.index('s')] = 0
    elif 'p' in i:
        i[i.index('p')] = 1
    elif 'd' in i:
        i[i.index('d')] = 2
    elif 'f' in i:
        i[i.index('f')] = 3
    elif 'g' in i:
        i[i.index('g')] = 4

#changes n element into integer, whether its 1 or 2 digits
for i in all_names:
    if isinstance(i[1], str): #means it hasnt been changed to 0,1,2 which means its a 2 digit n value. i.e. 12
        i[0] = int(i[0] + i[1])
        del i[1]
    else:
        i[0] = int(i[0])

#converts last 3 numbers into 1 j value
for i in all_names:
    i[2] = int(i[2]) / int(i[4])
    del i[4]
    del i[3]

#adds back in matrix elements and uncertainty, starts column titles
for count, i in enumerate(all_names):
    i.append(mats.matrix[count])
    i.append(mats.unc[count])
columns = ['n','l','j','m','unc']
starts = pd.DataFrame(all_names, columns=columns)

In [8]:
starts

,n,l,j,m,unc
0,6,0,0.5,4.5350435,0.0768256
1,6,0,0.5,6.3818104,0.0788595
2,6,0,0.5,0.2983429,0.0225619
3,6,0,0.5,0.6012643,0.0282228
4,6,0,0.5,0.0915939,0.0126111
...,...,...,...,...,...
643,13,2,2.5,161.1218600,3.7165400
644,13,2,2.5,61.4037750,0.7020830
645,13,2,2.5,274.6316500,3.1421300
646,13,2,2.5,3.7448749,0.4857401


In [9]:
all_names = []
#splits the original n+l number into components
for i in energies['level']:
    nm = split(i)
    name = [string for string in nm if string != " "]
    all_names.append(name)
 

#changes s, p, d into 0, 1, 2
for i in all_names:
    if 's' in i:
        i[i.index('s')] = 0
    elif 'p' in i:
        i[i.index('p')] = 1
    elif 'd' in i:
        i[i.index('d')] = 2
    elif 'f' in i:
        i[i.index('f')] = 3
    elif 'g' in i:
        i[i.index('g')] = 4

#changes n element into integer, whether its 1 or 2 digits
for i in all_names:
    if isinstance(i[1], str): #means it hasnt been changed to 0,1,2 which means its a 2 digit n value. i.e. 12
        i[0] = int(i[0] + i[1])
        del i[1]
    else:
        i[0] = int(i[0])

#converts last 3 numbers into 1 j value
for i in all_names:
    i[2] = int(i[2]) / int(i[4])
    del i[4]
    del i[3]

#adds back in matrix elements and uncertainty, starts column titles
for count, i in enumerate(all_names):
    i.append(energies.ene[count])
    i.append(energies.unc[count])
    i.append(energies.modif[count])
columns = ['n','l','j','ene','unc','modif']
n_ene = pd.DataFrame(all_names, columns=columns)
nist = n_ene.copy()

In [10]:
nist

,n,l,j,ene,unc,modif
0,6,0,0.5,0.0000,0.0000,None
1,7,0,0.5,18535.5286,0.0030,None
2,8,0,0.5,24317.1494,0.0000,None
3,9,0,0.5,26910.6627,0.0005,None
4,10,0,0.5,28300.2287,0.0005,None
5,11,0,0.5,29131.7300,0.0002,None
6,12,0,0.5,29668.8033,0.0002,None
7,13,0,0.5,30035.7883,0.0002,None
8,14,0,0.5,30297.6451,0.0002,None
9,6,1,0.5,11178.2681,0.0000,None


In [11]:
# Initial_en = []
# Final_en = []
# Initial_unc = []
# Final_unc = []
# decay = []
# hold = []
# for i in range(len(starts)):
#     for j in range(len(nist)):
#         if starts['n'][i] == nist['n'][j] and starts['j'][i] == nist['j'][j] and starts['l'][i] == nist['l'][j]:
#             Initial_en.append(nist['ene'][j])
#             Initial_unc.append(nist['unc'][j])
#             hold.append([[starts['n'][i],starts['l'][i],starts['j'][i]], starts['m'][i], starts['unc'][i],
#                          [ends['n'][i], ends['l'][i], ends['j'][i]],
#                          nist['ene'][j],nist['unc'][j]])
#         elif ends['n'][i] == nist['n'][j] and ends['j'][i] == nist['j'][j] and ends['l'][i] == nist['l'][j]:
#             Final_en.append(nist['ene'][j])
#             Final_unc.append(nist['unc'][j])
# hold = pd.DataFrame(hold, columns = ['Initial', 'matrix', 'mat_unc', 'Decay', 'Ei', 'Ei_unc'])
# hold['Ef'] = Final_en
# hold['Ef_unc'] = Final_unc
# columns = ['Initial', 'Decay', 'matrix', 'Ei', 'Ef', 'mat_unc', 'Ei_unc', 'Ef_unc']
# all_state = hold[columns]

# unc_hold = []
# for i in all_state['Ef_unc']:
#     unc_hold.append("{:.2e}".format(i))
# all_state['Ef_unc'] = unc_hold
# all_state['old_unc'] = database['unc']

In [12]:
Initial_en = []
Final_en = []
Initial_unc = []
Final_unc = []
decay = []
hold = []
for i in range(len(starts)):
    for j in range(len(nist)):
        if starts['n'][i] == nist['n'][j] and starts['j'][i] == nist['j'][j] and starts['l'][i] == nist['l'][j]:
            Initial_en.append(nist['ene'][j])
            Initial_unc.append(nist['unc'][j])
            hold.append([[starts['n'][i],starts['l'][i],starts['j'][i]], starts['m'][i], starts['unc'][i],
                         [ends['n'][i], ends['l'][i], ends['j'][i]],
                         nist['ene'][j],nist['unc'][j]])
        elif ends['n'][i] == nist['n'][j] and ends['j'][i] == nist['j'][j] and ends['l'][i] == nist['l'][j]:
            Final_en.append(nist['ene'][j])
            Final_unc.append(nist['unc'][j])
hold = pd.DataFrame(hold, columns = ['Initial', 'matrix', 'mat_unc', 'Decay', 'Ei', 'Ei_unc'])
hold['Ef'] = Final_en
hold['Ef_unc'] = Final_unc
columns = ['Initial', 'Decay', 'matrix', 'Ei', 'Ef', 'mat_unc', 'Ei_unc', 'Ef_unc']
all_state = hold[columns]

# unc_hold = []
# for i in all_state['Ef_unc']:
#     unc_hold.append("{:.2e}".format(i))
# all_state['Ef_unc'] = unc_hold
all_state['old_unc'] = database['unc']

<ipython-input-12-3aa04a282aa6>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_state['old_unc'] = database['unc']


In [13]:
all_state.Ei[0]

'0.0000'

In [14]:
#MAKING A CHANGE. OG: if all_state.Ei[i] < all_state.Ef[i]
eis = []
efs = []
inits = []
decs = []
eis_unc = []
efs_unc = []
for i in range(len(all_state)):
    if float(all_state.Ei[i]) < float(all_state.Ef[i]): #if we need to switch
        a = all_state.Ef[i]
        b = all_state.Ei[i]
        unc1 = all_state.Ef_unc[i]
        unc2 = all_state.Ei_unc[i]
        c2 = all_state.Decay[i]
        d = all_state.Initial[i]
        eis.append(a)
        eis_unc.append(unc1)
        efs.append(b)
        efs_unc.append(unc2)
        inits.append(c2)
        decs.append(d)

    else:
        a = all_state.Ei[i]
        b = all_state.Ef[i]
        unc1 = all_state.Ei_unc[i]
        unc2 = all_state.Ef_unc[i]
        c2 = all_state.Initial[i]
        d = all_state.Decay[i]
        eis.append(a)
        eis_unc.append(unc1)
        efs.append(b)
        efs_unc.append(unc2)
        inits.append(c2)
        decs.append(d)
        #print(unc1)
        
all_state['Initial'] = inits
all_state['Decay'] = decs
all_state['Ei'] = eis
all_state['Ef'] = efs
all_state['Ei_unc'] = eis_unc
all_state['Ef_unc'] = efs_unc
all_state.sort_values('Ei', inplace = True)
all_state.reset_index(inplace = True, drop = True)


<ipython-input-14-de4dc0adeb62>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_state['Initial'] = inits
<ipython-input-14-de4dc0adeb62>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_state['Decay'] = decs
<ipython-input-14-de4dc0adeb62>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#

In [15]:
initial_holds = list(all_state.Initial)
ends_holds = list(all_state.Decay)
mat_holds = list(all_state.matrix)
dups = []
for k, i in enumerate(all_state.matrix):
    ms = all_state.matrix[k]
    ini = all_state.Initial[k]
    end = all_state.Decay[k]
    res_list = [i for i in range(len(mat_holds)) if (mat_holds[i] == ms) 
                and (initial_holds[i] == ini) and (ends_holds[i] == end)]
    if len(res_list) > 1:
        dups.append(res_list[1:])
dups

[]

In [16]:
#Removes duplicates, resets index. 
if len(dups) != 0:
    dups = pd.DataFrame(dups)
    dups.drop_duplicates(inplace = True)
    dups = list(dups[0])
    all_state.drop(index = dups, inplace = True)
    all_state.reset_index(inplace = True, drop = True)
    all_state['matrix'] = pd.to_numeric(all_state.matrix)

In [17]:
#problem: branching ratio is Tr1/Sum(Trs), which is TR1 * lifetime. But lifetime changes

In [18]:
c_val = 2.02613 * 10**18 
def transition_rate_calc(m,j,lam): #takes in matrix element, total momentum, wavelength in cm-1
    lam = lam * 10**8 #converts to Angstrom from cm-1
    jval = 2 * j + 1 #bottom part of first half
    m2 = m**2 #matrix element squared. Will need to be variable
    lam3 = lam**3
    A = (c_val / jval) * (m2 / lam3)
    return A

def lifetime_calc(amas): #takes in list of all transition rates
    return 1/np.sum(amas)

def branching_ratio_calc(a, lifet): #takes in a transition rate
    return a*lifet

def energy_err_calc(Ei, Ef, Eierr, Eferr):
    a = Ei #higher energy
    b = Ef
    err_a = Eierr
    err_b = Eferr
    return (1/(a-b)**2) * np.sqrt( (err_a)**2 + (err_b)**2 ) * 10**8 #into Angstrom from cm-1
    
def transition_err_calc(m,j,lam, merr, lamerr): #takes in matrix element, total momentum, wavelength in Angstrom
    m, j, lam, merr, lamerr = m, j, lam, merr, lamerr
    m2 = m**2 #matrix element squared. Will need to be variable
    lam = lam * 10**8 #converting to Angstrom from cm -1. Lamerr is already converted.
    lam3 = lam**3
    jval = 2 * j + 1
    return (c_val / jval) * (m2/lam3) * np.sqrt( ((m2 * 2 * merr / m) / m2)**2 + ((lam3 * 3 * lamerr / lam) / lam3)**2 )

#Look into Transition error, right number, wrong factor
def lifetime_err_calc(trs, errs): #takes in list of all transition rates
    X = np.sum(trs)
    errs = [i ** 2 for i in errs]
    return (1 / X)**2 * np.sqrt(np.sum(errs))

def lifetime_err_calc2(trs, errs): #takes in list of all transition rates
    num_trans = len(errs) #number of rates
    errs = pd.DataFrame(errs, columns = ['n', 'l', 's', 'error'])
    errs.sort_values(by=['l', 'n', 's'], ascending = [True, True, True], inplace = True)
    errs.reset_index(drop = True, inplace = True)
    X = np.sum(trs)
    direct_add = [] #holder for individual sets of correlated states
    quad_add = [] #holder for sets which add in quadrature, non-correlated
    all_corr = [] #holds all the lists of errors for correlated states
    if num_trans > 1: #multiple transitions
        for i in range(len(errs)-1):
            if errs['n'][i] == errs['n'][i+1] and errs['l'][i] == errs['l'][i+1]:
                direct_add.append(errs['error'][i])
                direct_add.append(errs['error'][i+1])
            else:
                direct_add = list(dict.fromkeys(direct_add))
                all_corr.append(direct_add)
                direct_add = []
                quad_add.append(errs['error'][i])
                quad_add.append(errs['error'][i+1])
            
        direct_add = list(dict.fromkeys(direct_add)) #remove duplicates
        all_corr.append(direct_add) #added here in case there is no "else" case
        flat_all = [item for sublist in all_corr for item in sublist] #flattened all_corraleted
        flat_all = list(dict.fromkeys(flat_all)) #remove duplicate errors
        quad_add = list(dict.fromkeys(quad_add)) #remove duplicates
        quad_add = [elem for elem in quad_add if elem not in flat_all] #removed quadratic adding elements if they should be linear

        corr_sum = [] #directly added errors of correlated values
        for i in all_corr:
            corr_sum.append(np.sum(i))
        all_errs = corr_sum + quad_add
        squares = [i**2 for i in all_errs]
        #return np.sqrt(np.sum(squares))

        return (1 / X)**2 * np.sqrt(np.sum(squares))
    else:
        error = float(errs['error'])
        return (1 / X)**2 * np.sqrt(error**2)


def branching_ratio_error(Tr, Trs, TrError, TrErrors, lifetime):
    sums = 1/lifetime #sum of all transition rates
    all_errors = []
    for i in range(len(Trs)):
        if Trs[i] == Tr:
            numer = sums - Trs[i]
            denom = sums**2
            Error1 = (numer / denom)**2
            Error1 = Error1 * (TrErrors[i]**2)
        else:
            numer = Tr
            denom = sums**2
            Error1 = (numer / denom)**2
            Error1 = Error1 * (TrErrors[i]**2)
        all_errors.append(Error1)
    Br_error = np.sqrt(np.sum(all_errors))
    if len(all_errors) == 1:
        Br_error = 0
    return Br_error

def ang_to_nm(x):
    return x*(10**-1)
def cm_to_nm(x):
    return x*(10**7)
def cm_to_ang(x):
    return x*(10**8)
def s_to_ns(x):
    return x*(10**9)

def conversion(x):
    #return np.format_float_scientific(x, 3)
    return "{:.3e}".format(x)

In [19]:
# all_state['Ef_unc'].fillna(0, inplace = True)
# Ef_uncs = []
# for i in all_state.Ef_unc:
#     #print(i)
#     if i == 'nan':
#         Ef_uncs.append(0)
#     else:
#         Ef_uncs.append(i)
# #     try: 
# #         if np.isnan(i) == True or i == 'nan':
# #             print(i)
# #     except TypeError:
# #         if i == 'nan':
#             #print(i)

# all_state.Ef_unc = Ef_uncs
# all_state.Ei = all_state.Ei.apply(str).apply(Decimal).astype('float64')
# all_state.Ef = all_state.Ef.apply(str).apply(Decimal).astype('float64')
# all_state.Ei_unc = all_state.Ei_unc.apply(str).apply(Decimal).astype('float64')
# all_state.Ef_unc = all_state.Ef_unc.apply(str).apply(Decimal).astype('float64')
# all_state.mat_unc = all_state.mat_unc.apply(str).apply(Decimal).astype('float64')
# all_state.matrix = all_state.matrix.apply(str).apply(Decimal).astype('float64')
#save the string versions in separate list, put them back in post doing the transition rate calculations

In [20]:
all_state['Ef_unc'].fillna(0, inplace = True)
Ef_uncs = []
for i in all_state.Ef_unc:
    #print(i)
    if i == 'nan':
        Ef_uncs.append(0)
    else:
        Ef_uncs.append(i)
#     try: 
#         if np.isnan(i) == True or i == 'nan':
#             print(i)
#     except TypeError:
#         if i == 'nan':
            #print(i)

all_state.Ef_unc = Ef_uncs
all_state.Ei = all_state.Ei.apply(str).apply(Decimal)
all_state.Ef = all_state.Ef.apply(str).apply(Decimal)
all_state.Ei_unc = all_state.Ei_unc.apply(str).apply(Decimal)
all_state.Ef_unc = all_state.Ef_unc.apply(str).apply(Decimal)
all_state.mat_unc = all_state.mat_unc.apply(str).apply(Decimal)
all_state.matrix = all_state.matrix.apply(str).apply(Decimal)

d:\python3\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
d:\python3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [21]:
#creates new column mat_werr that has the matrix plus (#) format of error in one column
mat_werr = []
for i in range(len(all_state)):
    try:
        mat_werr.append(round(str(all_state.matrix[i]), str(all_state.mat_unc[i]), format = 'Drake'))
    except ValueError:
        print(i)
all_state['mat_werr'] = mat_werr

<ipython-input-21-8eb4f3f48750>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_state['mat_werr'] = mat_werr


In [22]:
all_state

,Initial,Decay,matrix,Ei,Ef,mat_unc,Ei_unc,Ef_unc,old_unc,mat_werr
0,"[6, 1, 0.5]","[6, 0, 0.5]",4.5350435,11178.2681,0.0000,0.0768256,0.0000,0.0000,77,4.535(77)
1,"[6, 1, 1.5]","[6, 0, 0.5]",6.3818104,11732.3071,0.0000,0.0788595,0.0000,0.0000,79,6.382(79)
2,"[5, 2, 1.5]","[6, 1, 1.5]",3.1872287,14499.2568,11732.3071,0.0681474,0.0010,0.0000,68,3.187(68)
3,"[5, 2, 1.5]","[6, 1, 0.5]",7.0634865,14499.2568,11178.2681,0.1359779,0.0010,0.0000,14,7.06(14)
4,"[5, 2, 2.5]","[6, 1, 1.5]",9.6585456,14596.8423,11732.3071,0.1849809,0.0002,0.0000,18,9.66(18)
...,...,...,...,...,...,...,...,...,...,...
643,"[12, 3, 2.5]","[6, 2, 1.5]",0.7977554,30640.3202,22588.8210,0.0180302,0.0001,0.0010,18,0.798(18)
644,"[12, 3, 2.5]","[6, 2, 2.5]",0.2190609,30640.3202,22631.6863,0.0045102,0.0001,0.0010,45,0.2191(45)
645,"[12, 3, 2.5]","[12, 2, 1.5]",2.5863489,30640.3202,30196.7963,0.8533215,0.0001,0.0005,85,2.59(85)
646,"[12, 3, 2.5]","[8, 2, 1.5]",0.9613422,30640.3202,27811.2400,0.0941597,0.0001,0.0005,94,0.961(94)


In [23]:
all_state[['n','l', 's']] = pd.DataFrame(all_state.Initial.tolist(), index= all_state.index)
all_state.sort_values(by=['l', 'n', 's'], ascending = [True, True, True], inplace = True)
all_state.reset_index(drop = True, inplace = True)

d:\python3\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
<ipython-input-23-adca0f0e1d6b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_state.sort_values(by=['l', 'n', 's'], ascending = [True, True, True], inplace = True)


In [24]:
#Error all quadrature VERSion (original)
# states = list(all_state.Initial)
# MatrixErrors, WavelengthsCm, WavelengthsUncAng, TransitionRates = [], [], [], []
# TransitionRateErrors, TransitionsForLifetime, TerrorsForLifetime, Lifetimes, LifetimeErrors = [], [], [], [], []
# BranchingRatios, BranchingRatioErrors = [], []
# for i in range(len(all_state)):
#     Ei = all_state.Ei[i]
#     Ef = all_state.Ef[i]
#     Eierr = all_state.Ei_unc[i]
#     Eferr = all_state.Ef_unc[i]

#     m = all_state.matrix[i]
#     j = all_state.Initial[i][2]
#     lam = 1 / (Ei - Ef)

#     #d = decimal.Decimal(str(all_state.matrix[i])) #how many decimals spots to go out to
#     #d = -1 * d.as_tuple().exponent
#     #merr = all_state.mat_unc[i] / (10 ** d)
#     merr = all_state.mat_unc[i]
    
#     lamerr = energy_err_calc(Ei, Ef, Eierr, Eferr)
#     TR = transition_rate_calc(m,j,lam)
    
#     TRerr = transition_err_calc(m,j,lam,merr,lamerr)
    
#     MatrixErrors.append(merr)
#     WavelengthsCm.append(lam)
#     WavelengthsUncAng.append(lamerr)

#     TransitionRates.append(TR)
#     TransitionRateErrors.append(TRerr)
    
#     n_dec = all_state.Decay[i][0]
#     l_dec = all_state.Decay[i][1]
#     s_dec = all_state.Decay[i][2]
#     TransitionsForLifetime.append(TR)
#     TerrorsForLifetime.append(TRerr)
    
#     try:
#         if all_state.Initial[i] not in states[i+1:]: #If next state NOT have same Initial State Name as the current one
#             Lftime = lifetime_calc(TransitionsForLifetime)
#             LftimeError = lifetime_err_calc(TransitionsForLifetime, TerrorsForLifetime)
            
#             Lifetimes.append((all_state.Initial[i], Lftime, LftimeError))
#             for i in range(0, len(TransitionsForLifetime)):
                
#                 BR = branching_ratio_calc(TransitionsForLifetime[i], Lftime)
#                 BRerr = branching_ratio_error(TransitionsForLifetime[i], TransitionsForLifetime, 
#                                              TerrorsForLifetime[i], TerrorsForLifetime, Lftime)
#                 BranchingRatios.append(BR)
#                 BranchingRatioErrors.append(BRerr)
                
                
#             TransitionsForLifetime = []
#             TerrorsForLifetime = [] #reset for next initial state
#     except KeyError:
#         print(i)
# all_state['wavelength'] = WavelengthsCm
# all_state['Eerr'] = WavelengthsUncAng
# all_state['transition_rate s-1'] = TransitionRates
# all_state['Terr'] = TransitionRateErrors
# #all_state['lifetimes'] = Lifetimes
# #all_state['Lerr'] = LifetimeErrors
# all_state['branching ratio'] = BranchingRatios
# all_state['Berr'] = BranchingRatioErrors
# all_state
# life_quad = Lifetimes.copy()
# all_quad = all_state.copy()

In [25]:
#Errors Linear + quadrature VERSion THAT WILL TRY HAVING LIFETiMES SEPARATE
#linear
# states = list(all_state.Initial)
# MatrixErrors, WavelengthsCm, WavelengthsUncAng, TransitionRates = [], [], [], []
# TransitionRateErrors, TransitionsForLifetime, TerrorsForLifetime, Lifetimes, LifetimeErrors = [], [], [], [], []
# BranchingRatios, BranchingRatioErrors = [], []
# for i in range(len(all_state)):
#     Ei = all_state.Ei[i]
#     Ef = all_state.Ef[i]
#     Eierr = all_state.Ei_unc[i]
#     Eferr = all_state.Ef_unc[i]

#     m = all_state.matrix[i]
#     j = all_state.Initial[i][2]
#     lam = 1 / (Ei - Ef)

#     #d = decimal.Decimal(str(all_state.matrix[i])) #how many decimals spots to go out to
#     #d = -1 * d.as_tuple().exponent
#     #merr = all_state.mat_unc[i] / (10 ** d)
#     merr = all_state.mat_unc[i]
    
#     lamerr = energy_err_calc(Ei, Ef, Eierr, Eferr)
#     TR = transition_rate_calc(m,j,lam)
    
#     TRerr = transition_err_calc(m,j,lam,merr,lamerr)
    
#     MatrixErrors.append(merr)

#     WavelengthsCm.append(lam)
#     WavelengthsUncAng.append(lamerr)

#     TransitionRates.append(TR)
#     TransitionRateErrors.append(TRerr)
    
#     n_dec = all_state.Decay[i][0]
#     l_dec = all_state.Decay[i][1]
#     s_dec = all_state.Decay[i][2]
#     TransitionsForLifetime.append(TR)
    
# #     TerrorsForLifetime.append(n_dec)
# #     TerrorsForLifetime.append(l_dec)
# #     TerrorsForLifetime.append(s_dec)
# #     TerrorsForLifetime.append(TRerr)
#     TerrorsForLifetime.append((n_dec, l_dec, s_dec, TRerr))
    
#     try:
#         if all_state.Initial[i] not in states[i+1:]: #If next state NOT have same Initial State Name as the current one
#             Lftime = lifetime_calc(TransitionsForLifetime)
#             LftimeError = lifetime_err_calc2(TransitionsForLifetime, TerrorsForLifetime)
            
#             Lifetimes.append((all_state.Initial[i], Lftime, LftimeError))
#             for i in range(0, len(TransitionsForLifetime)):
                
#                 BR = branching_ratio_calc(TransitionsForLifetime[i], Lftime)
#                 BRerr = branching_ratio_error(TransitionsForLifetime[i], TransitionsForLifetime, 
#                                                   TerrorsForLifetime[i][3], [trerr[3] for trerr in TerrorsForLifetime], Lftime)
#                 BranchingRatios.append(BR)
#                 BranchingRatioErrors.append(BRerr)
                
                
#             TransitionsForLifetime = []
#             TerrorsForLifetime = [] #reset for next initial state
#     except KeyError:
#         print(i)
        
# all_state['wavelength'] = WavelengthsCm
# all_state.wavelength = all_state.wavelength.apply(cm_to_nm)
# all_state['Eerr'] = WavelengthsUncAng
# all_state.Eerr = all_state.Eerr.apply(ang_to_nm)
# all_state['transition_rate s-1'] = TransitionRates
# all_state['Terr'] = TransitionRateErrors
# all_state['branching ratio'] = BranchingRatios
# all_state['Berr'] = BranchingRatioErrors
# life_linear = Lifetimes.copy()
# all_linear = all_state.copy()

In [26]:
#creates new all_state columns with Initial, Decay in '7s1/2' format
from sympy import pretty_print as pp, latex
from sympy import Symbol

ini_hold = []
dec_hold = []
n_holdI, l_holdI, s_holdI = [], [], []
n_holdD, l_holdD, s_holdD = [], [], []
for i in range(len(all_state)):
    #Initial
    n = str(all_state.Initial[i][0])
    if all_state.Initial[i][1] == 0:
        l = 's'
    elif all_state.Initial[i][1] == 1:
        l = 'p'
    elif all_state.Initial[i][1] == 2:
        l = 'd'
    elif all_state.Initial[i][1] == 3:
        l = 'f'
    if all_state.Initial[i][2] == 0.5:
        s = '1/2'
    elif all_state.Initial[i][2] == 1.5:
        s = '3/2'
    elif all_state.Initial[i][2] == 2.5:
        s = '5/2'
    elif all_state.Initial[i][2] == 3.5:
        s = '7/2'
    elif all_state.Initial[i][2] == 4.5:
        s = '9/2'
    ini = n+l+s
    ini_hold.append(ini)
    n_holdI.append(n)
    l_holdI.append(l)
    s_holdI.append(s)
    
    #Decay
    n = str(all_state.Decay[i][0])
    if all_state.Decay[i][1] == 0:
        l = 's'
    elif all_state.Decay[i][1] == 1:
        l = 'p'
    elif all_state.Decay[i][1] == 2:
        l = 'd'
    elif all_state.Decay[i][1] == 3:
        l = 'f'
    if all_state.Decay[i][2] == 0.5:
        s = '1/2'
    elif all_state.Decay[i][2] == 1.5:
        s = '3/2'
    elif all_state.Decay[i][2] == 2.5:
        s = '5/2'
    elif all_state.Decay[i][2] == 3.5:
        s = '7/2'
    elif all_state.Decay[i][2] == 4.5:
        s = '9/2'
    dec = n+l+s
    dec_hold.append(dec)
    n_holdD.append(n)
    l_holdD.append(l)
    s_holdD.append(s)
    
all_state['Initial_form'] = ini_hold #formatted 
all_state['Decay_form'] = dec_hold

<ipython-input-26-f69b59997ce0>:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_state['Initial_form'] = ini_hold #formatted
<ipython-input-26-f69b59997ce0>:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_state['Decay_form'] = dec_hold


In [27]:
#replaces exisiting values with experimental ones if available
exp = pd.read_csv('Experimental_Data\Cs-matrix-elements.csv') #experiment
a = list(all_state.Initial_form)
b = list(all_state.Decay_form)
c = list(zip(a,b))

d = list(exp.From)
e = list(exp.To)
f = list(zip(d,e))

#make list of indices, transitions rates, for me and Safronova. 
comparison = []
for i in range(len(c)):
    try:
        all_state.iloc[i, all_state.columns.get_loc('matrix')] = exp['value'][f.index(c[i])]
        all_state.iloc[i, all_state.columns.get_loc('mat_unc')] = exp['uncertainity'][f.index(c[i])]
        all_state.iloc[i, all_state.columns.get_loc('mat_werr')] = exp['Matrixelement'][f.index(c[i])] + '*'
#         comparison.append((i, f.index(c[i]), all_state['transition_rate (s-1)'][i], exp['value'][f.index(c[i])], 
#                           exp['uncertainity'][f.index(c[i])]))
    except ValueError:
        pass
all_state.drop(['Initial_form', 'Decay_form'], axis = 1, inplace = True)

d:\python3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
d:\python3\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [28]:
states = list(all_state.Initial)
MatrixErrors, WavelengthsCm, WavelengthsUncAng, TransitionRates = [], [], [], []
TransitionRateErrors, TransitionsForLifetime, TerrorsForLifetime, Lifetimes, LifetimeErrors = [], [], [], [], []
BranchingRatios, BranchingRatioErrors = [], []
for i in range(len(all_state)):
    Ei = all_state.Ei[i]
    Ef = all_state.Ef[i]
    Eierr = all_state.Ei_unc[i]
    Eferr = all_state.Ef_unc[i]

    m = float(all_state.matrix[i])
    j = float(all_state.Initial[i][2])
    lam = float(1 / (Ei - Ef))

    #d = decimal.Decimal(str(all_state.matrix[i])) #how many decimals spots to go out to
    #d = -1 * d.as_tuple().exponent
    #merr = all_state.mat_unc[i] / (10 ** d)
    merr = float(all_state.mat_unc[i])
    
    lamerr = float(energy_err_calc(Ei, Ef, Eierr, Eferr))
    TR = transition_rate_calc(m,j,lam)
    
    TRerr = transition_err_calc(m,j,lam,merr,lamerr)
    
    MatrixErrors.append(merr)

    WavelengthsCm.append(lam)
    WavelengthsUncAng.append(lamerr)

    TransitionRates.append(TR)
    TransitionRateErrors.append(TRerr)
    
    n_dec = all_state.Decay[i][0]
    l_dec = all_state.Decay[i][1]
    s_dec = all_state.Decay[i][2]
    TransitionsForLifetime.append(TR)
    
#     TerrorsForLifetime.append(n_dec)
#     TerrorsForLifetime.append(l_dec)
#     TerrorsForLifetime.append(s_dec)
#     TerrorsForLifetime.append(TRerr)
    TerrorsForLifetime.append((n_dec, l_dec, s_dec, TRerr))
    
    try:
        if all_state.Initial[i] not in states[i+1:]: #If next state NOT have same Initial State Name as the current one
            Lftime = lifetime_calc(TransitionsForLifetime)
            LftimeError = lifetime_err_calc2(TransitionsForLifetime, TerrorsForLifetime)
            
            Lifetimes.append((all_state.Initial[i], Lftime, LftimeError))
            for i in range(0, len(TransitionsForLifetime)):
                
                BR = branching_ratio_calc(TransitionsForLifetime[i], Lftime)
                BRerr = branching_ratio_error(TransitionsForLifetime[i], TransitionsForLifetime, 
                                                  TerrorsForLifetime[i][3], [trerr[3] for trerr in TerrorsForLifetime], Lftime)
                BranchingRatios.append(BR)
                BranchingRatioErrors.append(BRerr)
                
                
            TransitionsForLifetime = []
            TerrorsForLifetime = [] #reset for next initial state
    except KeyError:
        print(i)
        
all_state['wavelength'] = WavelengthsCm
all_state.wavelength = all_state.wavelength.apply(cm_to_nm)
all_state['Eerr'] = WavelengthsUncAng
all_state.Eerr = all_state.Eerr.apply(ang_to_nm)
all_state['transition_rate s-1'] = TransitionRates
all_state['Terr'] = TransitionRateErrors
all_state['branching ratio'] = BranchingRatios
all_state['Berr'] = BranchingRatioErrors
life_linear = Lifetimes.copy()
all_linear = all_state.copy()

<ipython-input-28-cad6a314653a>:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_state['wavelength'] = WavelengthsCm
d:\python3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
<ipython-input-28-cad6a314653a>:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

In [29]:
#saves the wavelength and its error with the same number of digits past the decimal as the original initial energy had
new_wavelength = []
new_wave_error = []
for p in range(len(all_state)):
    getcontext().prec = 10
    num_digits = len(str(all_state.Ei[p]).split('.')[1]) #how many digits past the decimal spot
    new_wavelength.append(round(str(all_state.wavelength[p]), decimals = num_digits))
    new_wave_error.append(round(str(all_state.Eerr[p]), decimals = num_digits))

all_state['wavelength'] = new_wavelength
all_state['Eerr'] = new_wave_error

<ipython-input-29-256207120a84>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_state['wavelength'] = new_wavelength
<ipython-input-29-256207120a84>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_state['Eerr'] = new_wave_error


In [30]:
save_copy = all_state.copy() #used to display error formatted numbers in TR table on main page

In [31]:
#Removes 'f' States and above
for i in range(len(Lifetimes)-1,0,-1): #counting backwards
    if Lifetimes[i][0][1] >= 3:
        del(Lifetimes[i])
for i in range(len(save_copy)-1, 0, -1):
    if save_copy.Initial[i][1] >= 3:
        save_copy.drop(i, axis = 0, inplace = True)

In [32]:
save_copy

,Initial,Decay,matrix,Ei,Ef,mat_unc,Ei_unc,Ef_unc,old_unc,mat_werr,n,l,s,wavelength,Eerr,transition_rate s-1,Terr,branching ratio,Berr
0,"[7, 0, 0.5]","[6, 1, 0.5]",4.2433832,18535.5286,11178.2681,0.0121211,0.0030,0.0000,12,4.243(12),7,0,0.5,1359.2016,0.0006,7.264575e+06,41502.093473,0.351673,1.854448e-03
1,"[7, 0, 0.5]","[6, 1, 1.5]",6.4795087,18535.5286,11732.3071,0.0187563,0.0030,0.0000,19,6.480(19),7,0,0.5,1469.8919,0.0006,1.339261e+07,77535.465598,0.648327,1.854448e-03
2,"[8, 0, 0.5]","[6, 1, 0.5]",1.0225182,24317.1494,11178.2681,0.0104242,0.0000,0.0000,10,1.023(10),8,0,0.5,761.0998,0.0000,2.402451e+06,48984.226709,0.222241,4.018874e-03
3,"[8, 0, 0.5]","[6, 1, 1.5]",1.4554009,24317.1494,11732.3071,0.0153247,0.0000,0.0000,15,1.455(15),8,0,0.5,794.6067,0.0000,4.277066e+06,90071.057345,0.395654,5.433871e-03
4,"[8, 0, 0.5]","[7, 1, 0.5]",9.3125109,24317.1494,21765.3480,0.0229434,0.0000,0.0200,23,9.313(23),8,0,0.5,3918.8003,0.0307,1.459861e+06,7193.453430,0.135046,1.440569e-03
5,"[8, 0, 0.5]","[7, 1, 1.5]",14.0659020,24317.1494,21946.3970,0.0677740,0.0000,0.0260,68,14.066(68),8,0,0.5,4218.0702,0.0463,2.670737e+06,25737.072172,0.247059,2.954886e-03
6,"[9, 0, 0.5]","[7, 1, 0.5]",1.9590677,26910.6627,21765.3480,0.0179181,0.0005,0.0200,18,1.959(18),9,0,0.5,1943.5157,0.0076,5.296293e+05,9688.234069,0.088585,1.700823e-03
7,"[9, 0, 0.5]","[7, 1, 1.5]",2.7273306,26910.6627,21946.3970,0.0277719,0.0005,0.0260,28,2.727(28),9,0,0.5,2014.3966,0.0106,9.218875e+05,18774.823729,0.154193,3.006409e-03
8,"[9, 0, 0.5]","[8, 1, 0.5]",16.0582220,26910.6627,25708.8354,0.0365920,0.0005,0.0000,37,16.058(37),9,0,0.5,8320.6630,0.0035,4.534803e+05,2066.698494,0.075848,7.988432e-04
9,"[9, 0, 0.5]","[6, 1, 0.5]",0.5448208,26910.6627,11178.2681,0.0056699,0.0005,0.0000,57,0.5448(57),9,0,0.5,635.6311,0.0000,1.170925e+06,24371.422490,0.195847,3.699554e-03


In [33]:
# holder = []
# for p in range(len(all_state)):
#     ini = all_state.Initial[p]
#     end = all_state.Decay[p]
#     value_wl = round(str(all_state.wavelength[p]), str(all_state.Eerr[p]), format = "Drake", err_dig = 1) #err_dig has to be after "Drake"
#     #value_tr = round(str(all_state['transition_rate s-1'][p]), str(all_state.Terr[p]), format = "Drake")
#     #value_br = round(str(all_state['branching ratio'][p]), str(all_state.Berr[p]), format = "Drake")
#     holder.append((p, ini, end, all_state.wavelength[p], all_state.Eerr[p], value_wl))
# holder

In [34]:
#Checks to make sure that wavelength has either no error () or 1 digit
# for i in range(len(holder)):
#     try:
#         length = len(holder[i][5].split('(')[1].split(')')[0])
#     except IndexError:
#         length = 0
#     if length > 1:
#         print(length)

In [35]:
# holder = []
# for p in range(len(all_state)):
#     ini = all_state.Initial[p]
#     end = all_state.Decay[p]
#     #value_wl = round(str(all_state.wavelength[p]), str(all_state.Eerr[p]), format = "Drake", err_dig = 1) #err_dig has to be after "Drake"
#     value_tr = round(str(all_state['transition_rate s-1'][p]), str(all_state.Terr[p]), format = "Drake")
#     #value_br = round(str(all_state['branching ratio'][p]), str(all_state.Berr[p]), format = "Drake")
#     holder.append((p, ini, end, all_state['transition_rate s-1'][p], all_state.Terr[p], value_tr))
# holder

In [36]:
holder = []
for p in range(len(save_copy)):
    ini = save_copy.Initial[p]
    end = save_copy.Decay[p]
    value_wl = round(str(save_copy.wavelength[p]), str(save_copy.Eerr[p]), format = "Drake", err_dig = 1)
    value_tr = round(str(save_copy['transition_rate s-1'][p]), str(save_copy.Terr[p]), format = "Drake")
    value_br = round(str(save_copy['branching ratio'][p]), str(save_copy.Berr[p]), format = "Drake")
    holder.append((value_wl, value_tr, value_br))
save_copy['wavelength'] = [i[0] for i in holder]
save_copy['transition_rate s-1'] = [i[1] for i in holder]
save_copy['branching ratio'] = [i[2] for i in holder]

C:\Users\dmgame\Documents\SafronovaResearch\LifetimesWebsite\modsigfig.py:572: UserWarning: warning: 2 significant figures requested from number with only 1 significant figures
  warn("warning: %d significant figures requested from number with only %d significant figures" % (given['sigfigs'], len(num.map)))


In [37]:
save_copy

,Initial,Decay,matrix,Ei,Ef,mat_unc,Ei_unc,Ef_unc,old_unc,mat_werr,n,l,s,wavelength,Eerr,transition_rate s-1,Terr,branching ratio,Berr
0,"[7, 0, 0.5]","[6, 1, 0.5]",4.2433832,18535.5286,11178.2681,0.0121211,0.0030,0.0000,12,4.243(12),7,0,0.5,1359.2016(6),0.0006,7.265(42)E+06,41502.093473,0.3517(19),1.854448e-03
1,"[7, 0, 0.5]","[6, 1, 1.5]",6.4795087,18535.5286,11732.3071,0.0187563,0.0030,0.0000,19,6.480(19),7,0,0.5,1469.8919(6),0.0006,1.3393(78)E+07,77535.465598,0.6483(19),1.854448e-03
2,"[8, 0, 0.5]","[6, 1, 0.5]",1.0225182,24317.1494,11178.2681,0.0104242,0.0000,0.0000,10,1.023(10),8,0,0.5,761.0998,0.0000,2.402(49)E+06,48984.226709,0.2222(40),4.018874e-03
3,"[8, 0, 0.5]","[6, 1, 1.5]",1.4554009,24317.1494,11732.3071,0.0153247,0.0000,0.0000,15,1.455(15),8,0,0.5,794.6067,0.0000,4.277(90)E+06,90071.057345,0.3957(54),5.433871e-03
4,"[8, 0, 0.5]","[7, 1, 0.5]",9.3125109,24317.1494,21765.3480,0.0229434,0.0000,0.0200,23,9.313(23),8,0,0.5,3918.80(3),0.0307,1.4599(72)E+06,7193.453430,0.1350(14),1.440569e-03
5,"[8, 0, 0.5]","[7, 1, 1.5]",14.0659020,24317.1494,21946.3970,0.0677740,0.0000,0.0260,68,14.066(68),8,0,0.5,4218.07(5),0.0463,2.671(26)E+06,25737.072172,0.2471(30),2.954886e-03
6,"[9, 0, 0.5]","[7, 1, 0.5]",1.9590677,26910.6627,21765.3480,0.0179181,0.0005,0.0200,18,1.959(18),9,0,0.5,1943.516(8),0.0076,5.296(97)E+05,9688.234069,0.0886(17),1.700823e-03
7,"[9, 0, 0.5]","[7, 1, 1.5]",2.7273306,26910.6627,21946.3970,0.0277719,0.0005,0.0260,28,2.727(28),9,0,0.5,2014.397(1),0.0106,9.22(19)E+05,18774.823729,0.1542(30),3.006409e-03
8,"[9, 0, 0.5]","[8, 1, 0.5]",16.0582220,26910.6627,25708.8354,0.0365920,0.0005,0.0000,37,16.058(37),9,0,0.5,8320.663(4),0.0035,4.535(21)E+05,2066.698494,0.07585(80),7.988432e-04
9,"[9, 0, 0.5]","[6, 1, 0.5]",0.5448208,26910.6627,11178.2681,0.0056699,0.0005,0.0000,57,0.5448(57),9,0,0.5,635.6311,0.0000,1.171(24)E+06,24371.422490,0.1958(37),3.699554e-03


In [38]:
#At this point:
#Wavelength and Error are in nm
#Transition Rate and Error are in s-1
#

In [39]:
duplicated_error = all_state[all_state.duplicated(['Terr'])]
if len(duplicated_error) == 0:
    print('No duplicated Errors, can run error calc2')
else:
    print('There are duplicated Transition rate Errors')
    raise SystemExit(0)

No duplicated Errors, can run error calc2


In [40]:
import decimal

# create a new context for this task
ctx = decimal.Context()

# 20 digits should be enough for everyone :D
ctx.prec = 30

def float_to_str(f):
    """
    Convert the given float to a string,
    without resorting to scientific notation
    """
    d1 = ctx.create_decimal(repr(f))
    return format(d1, 'f')

In [41]:
#Copy going into excel. Keeps States, matrix, wavelength, transition_rate, branching ratio with erros. 
#save_copy = all_state.copy()
save_copy.fillna(0, inplace = True)
save_copy[0:10]

#These are important to put in before saving for formatting
#save_copy['transition_rate s-1'] = save_copy['transition_rate s-1'].map(lambda x: '{:.3E}'.format(float(x)))
#save_copy['lifetimes'] = save_copy['lifetimes'].map(lambda x: '{:.3E}'.format(float(x)))

#save_copy['Eerr'] = save_copy['Eerr'].apply(ang_to_nm)
#save_copy['lifetimes'] = save_copy['lifetimes'].apply(s_to_ns)
#save_copy['Lerr'] = save_copy['Lerr'].apply(s_to_ns)


,Initial,Decay,matrix,Ei,Ef,mat_unc,Ei_unc,Ef_unc,old_unc,mat_werr,n,l,s,wavelength,Eerr,transition_rate s-1,Terr,branching ratio,Berr
0,"[7, 0, 0.5]","[6, 1, 0.5]",4.2433832,18535.5286,11178.2681,0.0121211,0.0030,0.0000,12,4.243(12),7,0,0.5,1359.2016(6),0.0006,7.265(42)E+06,41502.093473,0.3517(19),0.001854
1,"[7, 0, 0.5]","[6, 1, 1.5]",6.4795087,18535.5286,11732.3071,0.0187563,0.0030,0.0000,19,6.480(19),7,0,0.5,1469.8919(6),0.0006,1.3393(78)E+07,77535.465598,0.6483(19),0.001854
2,"[8, 0, 0.5]","[6, 1, 0.5]",1.0225182,24317.1494,11178.2681,0.0104242,0.0000,0.0000,10,1.023(10),8,0,0.5,761.0998,0.0000,2.402(49)E+06,48984.226709,0.2222(40),0.004019
3,"[8, 0, 0.5]","[6, 1, 1.5]",1.4554009,24317.1494,11732.3071,0.0153247,0.0000,0.0000,15,1.455(15),8,0,0.5,794.6067,0.0000,4.277(90)E+06,90071.057345,0.3957(54),0.005434
4,"[8, 0, 0.5]","[7, 1, 0.5]",9.3125109,24317.1494,21765.3480,0.0229434,0.0000,0.0200,23,9.313(23),8,0,0.5,3918.80(3),0.0307,1.4599(72)E+06,7193.453430,0.1350(14),0.001441
5,"[8, 0, 0.5]","[7, 1, 1.5]",14.0659020,24317.1494,21946.3970,0.0677740,0.0000,0.0260,68,14.066(68),8,0,0.5,4218.07(5),0.0463,2.671(26)E+06,25737.072172,0.2471(30),0.002955
6,"[9, 0, 0.5]","[7, 1, 0.5]",1.9590677,26910.6627,21765.3480,0.0179181,0.0005,0.0200,18,1.959(18),9,0,0.5,1943.516(8),0.0076,5.296(97)E+05,9688.234069,0.0886(17),0.001701
7,"[9, 0, 0.5]","[7, 1, 1.5]",2.7273306,26910.6627,21946.3970,0.0277719,0.0005,0.0260,28,2.727(28),9,0,0.5,2014.397(1),0.0106,9.22(19)E+05,18774.823729,0.1542(30),0.003006
8,"[9, 0, 0.5]","[8, 1, 0.5]",16.0582220,26910.6627,25708.8354,0.0365920,0.0005,0.0000,37,16.058(37),9,0,0.5,8320.663(4),0.0035,4.535(21)E+05,2066.698494,0.07585(80),0.000799
9,"[9, 0, 0.5]","[6, 1, 0.5]",0.5448208,26910.6627,11178.2681,0.0056699,0.0005,0.0000,57,0.5448(57),9,0,0.5,635.6311,0.0000,1.171(24)E+06,24371.422490,0.1958(37),0.003700


In [42]:
#has to use all_state because save_copy has been changed to nm
#I think this might run into an error if the wavelength/error is 0. make an exception case if so
#changes wavelength to () format
# wave_werr = []
# for i in range(len(save_copy)):
#     wavel = float_to_str(all_state['wavelength'][i]*10**7) #from cm to nm. 
#     wave_error = all_state['Eerr'][i]*10**-1 #from Angstrom to nm
#     #print(wavel, wave_error)
#     if wave_error >= 10:
#         Er = str(round(wave_error))
#         #print(Er)
#         wave_rounded = round(float(wavel)) #get to integer
# #         Er_figs = len(Er) - 4 #sig figs, removing .0 component
# #         num_figs = len(str(wave_rounded)) - len(Er) + 1
# #         N = "%%.%xE" % num_figs
# #         wave_rounded = N % round(all_state.wavelength[i]*10**8, 3)
# #         Er = str(round(float(Er), -Er_figs)) #rounding S
# #         Er = Er[0:2] #need to round it
# #         wave_werr.append(wave_rounded + ' (%s)' % Er[0:2])
#         wave_werr.append(str(wave_rounded) + '(%s)' % Er[0:2])
        
#     elif wave_error < 10 and wave_error >= 1.0:
#         Er = str(round(wave_error, 1))
#         Er = Er.replace('.', '')
#         wave_rounded = wavel.split('.')[0] + '.' + wavel.split('.')[1][0:2] #have to include the first 2 decimal points for rounding
#         wave_rounded = str(round(float(wave_rounded), 1))
#         wave_werr.append(str(round(float(wavel), 1)) + '(%s)' % Er[0:2])
#     else:
#         Er = '%E' % wave_error #error to use, in angstrom
#         Er = Er.replace('.','') #remove '.' from error
#         length = Er.split('E')[1] #the exponent component. How many zeroes it went back
#         length = int(np.abs(float(length))) #get just number of exponent component
#         Er = '%E' % round(float(str(wave_error)),length+1)
#         Er = Er.replace('.','')
#         wave_rounded = float('.' + wavel.split('.')[1][:length+2]) #decimal component of wavelength up to error length + 1
#         round_length = len( wavel.split('.')[1][:length+2])-1 #how far out need to round
#         N = "%%.%sf" % round_length 
#         wave_rounded = N % round(wave_rounded, round_length) #done this way so that trailing zeroes are included
#         wave_rounded = wavel.split('.')[0] + '.' + wave_rounded.split('.')[1] #left hand side of wavelenght, ., right hand side
#         wave_werr.append(wave_rounded + '(%s)' % Er[0:2])
# #wave_werr
# save_copy['wavelength'] = wave_werr

In [43]:
# branch_werr = []
# for i in range(len(save_copy)):
# #for i in range(294, 296):
#     branchr = float_to_str(all_state['branching ratio'][i]) #branchrength. In cm converted to angstrom
#     if all_state.Berr[i] >= 10:
#         Er = str(round(all_state.Berr[i])) #round to nearest integer
#         Er = Er[0:2] #need to round it
#         branch_rounded = branchr.split('.')[0] + '.' + branchr.split('.')[1][0]
#         branch_rounded = str(round(float(branch_rounded)))
#         branch_werr.append(branch_rounded + '(%s)' % Er[0:2])
#     elif all_state.Berr[i] < 10 and all_state.Berr[i] >= 1.0:
#         Er = str(round(all_state.Berr[i], 1))
#         Er = Er.replace('.', '')
#         branch_rounded = branchr.split('.')[0] + '.' + branchr.split('.')[1][0:2] #have to include the first 2 decimal points for rounding
#         branch_rounded = str(round(float(branch_rounded), 1))
#         branch_werr.append(str(round(float(branchr), 1)) + '(%s)' % Er[0:2])
#     else:
        
#         Er = '%E' % all_state.Berr[i] #error to use, in angstrom
#         Er = Er.replace('.','') #remove '.' from error
#         length = Er.split('E')[1] #the exponent component. How many zeroes it went back
#         length = int(np.abs(float(length))) #get just number of exponent component
#         Er = '%E' % round(float(str(all_state.Berr[i])),length+1)
#         Er = Er.replace('.','')
#         branch_rounded = float('.' + branchr.split('.')[1][:length+2]) #decimal component of branchrength up to error length + 1
#         round_length = len( branchr.split('.')[1][:length+2])-1 #how far out need to round
#         N = "%%.%sf" % round_length 
#         branch_rounded = N % round(branch_rounded, round_length) #done this way so that trailing zeroes are included
#         try:
#             branch_rounded = branchr.split('.')[0] + '.' + branch_rounded.split('.')[1] #left hand side of branchrenght, ., right hand side
#         except IndexError: #means error is 0, there is no [1] index
#             branch_rounded = branchr.split('.')[0] 
#         branch_werr.append(branch_rounded + '(%s)' % Er[0:2])
        
# #branch_werr
# save_copy['branching ratio'] = branch_werr

In [44]:
#transition rate to ()
# trans_werr = []
# for i in range(len(save_copy)):
#     #transr = str(all_state['transition_rate s-1'][i]) 
#     transr = float_to_str(all_state['transition_rate s-1'][i])
#     if all_state.Terr[i] >= 10:
#         Er = str(round(all_state.Terr[i])) #get to integer
#         trans_rounded = round(all_state['transition_rate s-1'][i]) #get to integer
#         Er_figs = len(Er) - 4 #sig figs, removing .0 component
#         num_figs = len(str(trans_rounded)) - len(Er) + 1
#         N = "%%.%xE" % num_figs
#         trans_rounded = N % round(all_state['transition_rate s-1'][i], 3)
#         Er = str(round(float(Er), -Er_figs)) #rounding S
#         Er = Er[0:2] #need to round it
#         trans_werr.append(trans_rounded.split('E')[0] + '(%s)' % Er[0:2] + 'E' + trans_rounded.split('E')[1])
#         #print(trans_rounded.split('E')[0] + '(%s)' % Er[0:2] + 'E' + trans_rounded.split('E')[1])
#     elif all_state.Terr[i] < 10 and all_state.Terr[i] >= 1.0:
#         Er = str(round(all_state.Terr[i], 1))
#         Er = Er.replace('.', '')
#         trans_rounded = transr.split('.')[0] + '.' + transr.split('.')[1][0:2] #have to include the first 2 decimal points for rounding
#         trans_rounded = str(round(float(trans_rounded), 1))
#         trans_werr.append(str(round(float(transr), 1)) + '(%s)' % Er[0:2])
#     else:
        
#         Er = '%E' % all_state.Terr[i] #error to use, in angstrom
#         Er = Er.replace('.','') #remove '.' from error
#         length = Er.split('E')[1] #the exponent component. How many zeroes it went back
#         length = int(np.abs(float(length))) #get just number of exponent component
#         Er = '%E' % round(float(str(all_state.Terr[i])),length+1)
#         Er = Er.replace('.','')
#         trans_rounded = float('.' + transr.split('.')[1][:length+2]) #decimal component of transrength up to error length + 1
#         round_length = len( transr.split('.')[1][:length+2])-1 #how far out need to round
#         N = "%%.%sf" % round_length 
#         trans_rounded = N % round(trans_rounded, round_length) #done this way so that trailing zeroes are included
#         trans_rounded = transr.split('.')[0] + '.' + trans_rounded.split('.')[1] #left hand side of transrenght, ., right hand side
#         trans_werr.append(trans_rounded + '(%s)' % Er[0:2])
# #trans_werr
# save_copy['transition_rate s-1'] = trans_werr

In [45]:
save_copy.drop(['Ei','Ef','mat_unc','Ei_unc','Ef_unc', 'Eerr', 'matrix', 'old_unc', 'Berr'], axis = 1, inplace = True)
save_copy[0:15]

,Initial,Decay,mat_werr,n,l,s,wavelength,transition_rate s-1,Terr,branching ratio
0,"[7, 0, 0.5]","[6, 1, 0.5]",4.243(12),7,0,0.5,1359.2016(6),7.265(42)E+06,41502.093473,0.3517(19)
1,"[7, 0, 0.5]","[6, 1, 1.5]",6.480(19),7,0,0.5,1469.8919(6),1.3393(78)E+07,77535.465598,0.6483(19)
2,"[8, 0, 0.5]","[6, 1, 0.5]",1.023(10),8,0,0.5,761.0998,2.402(49)E+06,48984.226709,0.2222(40)
3,"[8, 0, 0.5]","[6, 1, 1.5]",1.455(15),8,0,0.5,794.6067,4.277(90)E+06,90071.057345,0.3957(54)
4,"[8, 0, 0.5]","[7, 1, 0.5]",9.313(23),8,0,0.5,3918.80(3),1.4599(72)E+06,7193.453430,0.1350(14)
5,"[8, 0, 0.5]","[7, 1, 1.5]",14.066(68),8,0,0.5,4218.07(5),2.671(26)E+06,25737.072172,0.2471(30)
6,"[9, 0, 0.5]","[7, 1, 0.5]",1.959(18),9,0,0.5,1943.516(8),5.296(97)E+05,9688.234069,0.0886(17)
7,"[9, 0, 0.5]","[7, 1, 1.5]",2.727(28),9,0,0.5,2014.397(1),9.22(19)E+05,18774.823729,0.1542(30)
8,"[9, 0, 0.5]","[8, 1, 0.5]",16.058(37),9,0,0.5,8320.663(4),4.535(21)E+05,2066.698494,0.07585(80)
9,"[9, 0, 0.5]","[6, 1, 0.5]",0.5448(57),9,0,0.5,635.6311,1.171(24)E+06,24371.422490,0.1958(37)


In [46]:
#Changes numerical back to normal string names for Initial and Decay
from sympy import pretty_print as pp, latex
from sympy import Symbol

ini_hold = []
dec_hold = []
n_holdI, l_holdI, s_holdI = [], [], []
n_holdD, l_holdD, s_holdD = [], [], []
for i in range(len(save_copy)):
    #Initial
    n = str(save_copy.Initial[i][0])
    if save_copy.Initial[i][1] == 0:
        l = 's'
    elif save_copy.Initial[i][1] == 1:
        l = 'p'
    elif save_copy.Initial[i][1] == 2:
        l = 'd'
    elif save_copy.Initial[i][1] == 3:
        l = 'f'
    if save_copy.Initial[i][2] == 0.5:
        s = '1/2'
    elif save_copy.Initial[i][2] == 1.5:
        s = '3/2'
    elif save_copy.Initial[i][2] == 2.5:
        s = '5/2'
    elif save_copy.Initial[i][2] == 3.5:
        s = '7/2'
    elif save_copy.Initial[i][2] == 4.5:
        s = '9/2'
    ini = n+l+s
    ini_hold.append(ini)
    n_holdI.append(n)
    l_holdI.append(l)
    s_holdI.append(s)
    
    #Decay
    n = str(save_copy.Decay[i][0])
    if save_copy.Decay[i][1] == 0:
        l = 's'
    elif save_copy.Decay[i][1] == 1:
        l = 'p'
    elif save_copy.Decay[i][1] == 2:
        l = 'd'
    elif save_copy.Decay[i][1] == 3:
        l = 'f'
    if save_copy.Decay[i][2] == 0.5:
        s = '1/2'
    elif save_copy.Decay[i][2] == 1.5:
        s = '3/2'
    elif save_copy.Decay[i][2] == 2.5:
        s = '5/2'
    elif save_copy.Decay[i][2] == 3.5:
        s = '7/2'
    elif save_copy.Decay[i][2] == 4.5:
        s = '9/2'
    dec = n+l+s
    dec_hold.append(dec)
    n_holdD.append(n)
    l_holdD.append(l)
    s_holdD.append(s)
    
save_copy['Initial'] = ini_hold
save_copy['Decay'] = dec_hold
save_copy['nI'] = n_holdI
save_copy['lI'] = l_holdI
save_copy['sI'] = s_holdI
save_copy['nD'] = n_holdD
save_copy['lD'] = l_holdD
save_copy['sD'] = s_holdD
save_copy[0:26]

,Initial,Decay,mat_werr,n,l,s,wavelength,transition_rate s-1,Terr,branching ratio,nI,lI,sI,nD,lD,sD
0,7s1/2,6p1/2,4.243(12),7,0,0.5,1359.2016(6),7.265(42)E+06,41502.093473,0.3517(19),7,s,1/2,6,p,1/2
1,7s1/2,6p3/2,6.480(19),7,0,0.5,1469.8919(6),1.3393(78)E+07,77535.465598,0.6483(19),7,s,1/2,6,p,3/2
2,8s1/2,6p1/2,1.023(10),8,0,0.5,761.0998,2.402(49)E+06,48984.226709,0.2222(40),8,s,1/2,6,p,1/2
3,8s1/2,6p3/2,1.455(15),8,0,0.5,794.6067,4.277(90)E+06,90071.057345,0.3957(54),8,s,1/2,6,p,3/2
4,8s1/2,7p1/2,9.313(23),8,0,0.5,3918.80(3),1.4599(72)E+06,7193.453430,0.1350(14),8,s,1/2,7,p,1/2
5,8s1/2,7p3/2,14.066(68),8,0,0.5,4218.07(5),2.671(26)E+06,25737.072172,0.2471(30),8,s,1/2,7,p,3/2
6,9s1/2,7p1/2,1.959(18),9,0,0.5,1943.516(8),5.296(97)E+05,9688.234069,0.0886(17),9,s,1/2,7,p,1/2
7,9s1/2,7p3/2,2.727(28),9,0,0.5,2014.397(1),9.22(19)E+05,18774.823729,0.1542(30),9,s,1/2,7,p,3/2
8,9s1/2,8p1/2,16.058(37),9,0,0.5,8320.663(4),4.535(21)E+05,2066.698494,0.07585(80),9,s,1/2,8,p,1/2
9,9s1/2,6p1/2,0.5448(57),9,0,0.5,635.6311,1.171(24)E+06,24371.422490,0.1958(37),9,s,1/2,6,p,1/2


In [47]:
#save_copy['transition_rate s-1'] = save_copy['transition_rate s-1'].map(lambda x: '{:.3E}'.format(float(x)))
#save_copy['branching ratio'] = save_copy['branching ratio'].map(lambda x: '{:,.5f}'.format(float(x)))
#save_copy['wavelength'] = save_copy['wavelength'].map(lambda x: '{:,.3f}'.format(float(x)))

In [48]:
save_copy.drop(['nI', 'lI', 'sI', 'nD', 'sD', 'lD'], axis = 1, inplace = True)
save_copy = save_copy[['Initial','Decay','mat_werr','wavelength','transition_rate s-1', 'branching ratio']]
save_copy.rename(columns = {"mat_werr": "Matrix element (a.u.)", "wavelength": "Wavelength (nm)", 
                            "transition_rate s-1": "Transition Rate (s-1)", 
                            'branching ratio': "Branching ratio"}, inplace = True)
save_copy

d:\python3\lib\site-packages\pandas\core\frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,Initial,Decay,Matrix element (a.u.),Wavelength (nm),Transition Rate (s-1),Branching ratio
0,7s1/2,6p1/2,4.243(12),1359.2016(6),7.265(42)E+06,0.3517(19)
1,7s1/2,6p3/2,6.480(19),1469.8919(6),1.3393(78)E+07,0.6483(19)
2,8s1/2,6p1/2,1.023(10),761.0998,2.402(49)E+06,0.2222(40)
3,8s1/2,6p3/2,1.455(15),794.6067,4.277(90)E+06,0.3957(54)
4,8s1/2,7p1/2,9.313(23),3918.80(3),1.4599(72)E+06,0.1350(14)
5,8s1/2,7p3/2,14.066(68),4218.07(5),2.671(26)E+06,0.2471(30)
6,9s1/2,7p1/2,1.959(18),1943.516(8),5.296(97)E+05,0.0886(17)
7,9s1/2,7p3/2,2.727(28),2014.397(1),9.22(19)E+05,0.1542(30)
8,9s1/2,8p1/2,16.058(37),8320.663(4),4.535(21)E+05,0.07585(80)
9,9s1/2,6p1/2,0.5448(57),635.6311,1.171(24)E+06,0.1958(37)


In [49]:
#create safety, sort lifetimes
Life_holder = Lifetimes.copy()
Lifetimes = pd.DataFrame(Lifetimes, columns = ['State', 'Lifetime', 'Error'])

Lifetimes[['n','l', 's']] = pd.DataFrame(Lifetimes.State.tolist(), index= Lifetimes.index)
Lifetimes.sort_values(by=['l', 'n', 's'], ascending = [True, True, True], inplace = True)
Lifetimes.drop(['n','l','s'], axis = 1, inplace = True)
Lifetimes.reset_index(drop = True, inplace = True)

In [50]:
#change back to s, p, d
ini_hold = []
n_hold = []
l_hold = []
s_hold = []
for i in range(len(Lifetimes)):
    #State
    n = str(Lifetimes.State[i][0])
    if Lifetimes.State[i][1] == 0:
        l = 's'
    elif Lifetimes.State[i][1] == 1:
        l = 'p'
    elif Lifetimes.State[i][1] == 2:
        l = 'd'
    if Lifetimes.State[i][2] == 0.5:
        #s = '\u2081\u2082'
        #s = Symbol('_{1/2}')
        s = '1/2'
    elif Lifetimes.State[i][2] == 1.5:
        s = '3/2'
    elif Lifetimes.State[i][2] == 2.5:
        s = '5/2'
    ini = n+l+s
    
    n_hold.append(n)
    l_hold.append(l)
    s_hold.append(s)
    ini_hold.append(ini)
    
    
Lifetimes['State'] = ini_hold
Lifetimes['n'] = n_hold
Lifetimes['l'] = l_hold
Lifetimes['s'] = s_hold
Lifetimes.drop(['n','l','s'], axis = 1, inplace = True)
Lifetimes[0:10]

,State,Lifetime,Error
0,7s1/2,4.840930e-08,2.789598e-10
1,8s1/2,9.250596e-08,1.222857e-09
2,9s1/2,1.672582e-07,2.174201e-09
3,10s1/2,2.788139e-07,3.543244e-09
4,11s1/2,4.343549e-07,5.444544e-09
5,12s1/2,6.412375e-07,7.967087e-09
6,13s1/2,9.068684e-07,1.121014e-08
7,14s1/2,1.238668e-06,1.527102e-08
8,6p1/2,3.436182e-08,1.164208e-09
9,6p3/2,3.001612e-08,7.418135e-10


In [51]:
Lifetime_excel = Lifetimes.copy()

In [84]:
holder = []
for p in range(len(Lifetimes)):
    value_lt = round(str(Lifetimes.Lifetime[p]), str(Lifetimes.Error[p]), format = 'Drake')
    #print(p, value_lt)
    lifet_sci = "{:E}".format(Decimal(value_lt.split('(')[0])).split('E')[0] + '(' + value_lt.split('(')[1] + 'E' + "{:E}".format(Decimal(value_lt.split('(')[0])).split('E')[1]
    #holder.append((Lifetimes.Lifetime[p], Lifetimes.Error[p], lifet_sci))
    holder.append(lifet_sci)
Lifetimes['Lifetime'] = holder


C:\Users\dmgame\Documents\SafronovaResearch\LifetimesWebsite\modsigfig.py:572: UserWarning: warning: 2 significant figures requested from number with only 1 significant figures
  warn("warning: %d significant figures requested from number with only %d significant figures" % (given['sigfigs'], len(num.map)))


In [85]:
exp_l = pd.read_csv('Experimental_Data\Cs-lifetime.csv')

In [86]:
exp_l

,State,Value,Uncertainty,Lifetime,Ref
0,6p1/2,3.490000e-08,9.400000e-11,34.934(94)E-9,E27
1,6p3/2,3.050000e-08,4.600000e-11,30.462(46)E-9,E67
2,5d5/2,1.350000e-06,5.000000e-09,1353(5)E-9,E68
3,7p1/2,1.550000e-07,4.000000e-09,155(4)E-9,E51
4,7p3/2,1.330000e-07,2.000000e-09,133(2)E-9,E51


In [56]:
#WORK TO BE DONE: RENAME COLUMNS SO THIS WORKS WITH EVERY ELEMENT

In [88]:
exp_l = pd.read_csv('Experimental_Data\Cs-lifetime.csv') #experiment
a = list(Lifetimes.State)
#b = list(Lifetimes.Decay_form)
c = list(zip(a))

d = list(exp_l.State)
#e = list(exp.To)
f = list(zip(d))

#make list of indices, transitions rates, for me and Safronova. 
comparison = []
for i in range(len(c)):
    try:
        Lifetime_excel.iloc[i, Lifetime_excel.columns.get_loc('Lifetime')] = exp_l['Value'][f.index(c[i])]
        Lifetime_excel.iloc[i, Lifetime_excel.columns.get_loc('Error')] = exp_l['Uncertainty'][f.index(c[i])]
        Lifetimes.iloc[i, Lifetimes.columns.get_loc('Lifetime')] = exp_l['Lifetime'][f.index(c[i])] + '*'
#         comparison.append((i, f.index(c[i]), Lifetime['transition_rate (s-1)'][i], exp['value'][f.index(c[i])], 
#                           exp['uncertainity'][f.index(c[i])]))
    except ValueError:
        pass
Lifetimes.drop('Error', axis = 1, inplace = True)

In [89]:
# #changes Lifetime in () format
# life_werr = []
# for i in range(len(Lifetimes)):
# #for i in range(18, 22):
#     #lifet = str(Lifetimes.Lifetime[i])
#     lifet = float_to_str(Lifetimes.Lifetime[i]*10**9)
#     life_error = Lifetimes.Error[i]*10**9
#     #print(lifet, life_error)
#     if life_error >= 10:
#         Er = str(round(life_error)) #get to integer
#         life_rounded = round(float(lifet)) #get to integer
# #         Er_figs = len(Er) - 4 #sig figs, removing .0 component
# #         num_figs = len(str(life_rounded)) - len(Er) + 1
# #         print(num_figs, life_rounded, Er)
# #         N = "%%.%xE" % num_figs
# #         life_rounded = N % round(float(lifet), 3)
# #         Er = str(round(float(Er), -Er_figs)) #rounding S
# #         Er = Er[0:2] #need to round it
# #         life_werr.append(life_rounded + ' (%s)' % Er[0:2])
#         life_werr.append(str(life_rounded) + '(%s)' % Er[0:2])
        
        
# #         life_rounded = round(Lifetimes.Lifetime[i])
# #         Er_figs = len(Er) - 4 #sig figs, removing .0 component
# #         num_figs = len(str(life_rounded)) - len(Er) + 1
# #         N = "%%.%fE" % num_figs
# #         life_rounded = N % round(Lifetimes.Lifetime[i], 3)
# #         Er = str(round(float(Er), -Er_figs)) #rounding S
# #         Er = Er[0:2] #need to round it
# #         life_werr.append(life_rounded + ' (%s)' % Er[0:2])
        
#     elif life_error < 10 and life_error >= 1.0:
        
#         Er = str(round(life_error, 1))
#         Er = Er.replace('.', '')
#         life_rounded = lifet.split('.')[0] + '.' + lifet.split('.')[1][0:2] #have to include the first 2 decimal points for rounding
#         life_rounded = str(round(float(life_rounded), 1))
#         #print(str(round(float(lifet), 1)) + '(%s)' % Er[0:2], 'new')
#         life_werr.append(str(round(float(lifet), 1)) + '(%s)' % Er[0:2])
#     else:
#         Er = '%E' % life_error 
#         Er = Er.replace('.','') #remove '.' from error
#         length = Er.split('E')[1] #the exponent component. How many zeroes it went back
#         length = int(np.abs(float(length))) #get just number of exponent component
#         Er = '%E' % round(float(str(life_error)),length+1)
#         Er = Er.replace('.','')
#         life_rounded = float('.' + lifet.split('.')[1][:length+2]) #decimal component of wavelength up to error length + 1
#         round_length = len( lifet.split('.')[1][:length+2])-1 #how far out need to round
#         N = "%%.%sf" % round_length 
#         life_rounded = N % round(life_rounded, round_length) #done this way so that trailing zeroes are included
#         life_rounded = lifet.split('.')[0] + '.' + life_rounded.split('.')[1] #left hand side of wavelenght, ., right hand side
#         life_werr.append(life_rounded + '(%s)' % Er[0:2])
        
        
# #         Er = '%E' % Lifetimes.Error[i] #error to use, in angstrom
# #         Er = Er.replace('.','') #remove '.' from error
# #         length = Er.split('E')[1] #the exponent component. How many zeroes it went back
# #         length = int(np.abs(float(length))) #get just number of exponent component
# #         Er = '%E' % round(float(str(Lifetimes.Error[i])),length+1)
# #         Er = Er.replace('.','')
# #         life_rounded = float('.' + lifet.split('.')[1][:length+2]) #decimal component of lifetength up to error length + 1
# #         round_length = len( lifet.split('.')[1][:length+2])-1 #how far out need to round
# #         life_rounded = str(round(life_rounded, round_length))
# #         #life_rounded = N % round(trans_rounded, round_length)
# #         life_rounded = lifet.split('.')[0] + '.' + life_rounded.split('.')[1] #left hand side of lifetenght, ., right hand side
# #         life_werr.append(str(round(float(lifet), round_length)) + ' (%s)' % Er[0:2])
# #life_werr
# Lifetimes['Lifetime'] = life_werr

In [90]:
save_copy.to_csv('Transition_Formatted.txt', sep='\t', index = False)
Lifetimes.to_csv('Lifetime_Formatted.txt', sep='\t', index = False)

In [91]:
#excel copy is the version that will be saved when they click "excel"
excel_copy = all_state.copy()
for i in range(len(excel_copy)-1, 0, -1):
    if excel_copy.Initial[i][1] >= 3:
        excel_copy.drop(i, axis = 0, inplace = True)
excel_copy['Initial'] = save_copy['Initial']
excel_copy['Decay'] = save_copy['Decay']
excel_copy.drop(['Ei', 'Ef', 'Ei_unc', 'Ef_unc', 'mat_werr', 'old_unc', 'n', 'l', 's'], axis = 1, inplace = True)
excel_copy.rename(columns = {'matrix': 'Matrix Element (a.u.)', 
                             'mat_unc': 'Matrix Error', 'wavelength': 'Wavelength (nm)', 'Eerr': 'Wavelength Error',
                             'transition_rate s-1': 'Transition Rate (Hz)', 'Terr': 'Transition Rate Error',
                            'branching ratio': "Branching Ratio", 'Berr': "Branching Ratio Error"}, inplace = True)
#excel_copy['Wavelength (nm)'] = excel_copy['Wavelength (nm)']*10**7 #nm conversion
#excel_copy

In [92]:
#no_errors is the version that will be shown when they click "all"
no_errors = save_copy.copy()
matrixx = []
wavell = []
transrr = []
branchh = []
for i in range(len(no_errors)):
    matrixx.append(no_errors['Matrix element (a.u.)'][i].split('(')[0])
    wavell.append(no_errors['Wavelength (nm)'][i].split('(')[0])
    transrr.append(no_errors['Transition Rate (s-1)'][i].split('(')[0] + no_errors['Transition Rate (s-1)'][i].split(')')[1])
    branchh.append(no_errors['Branching ratio'][i].split('(')[0])
no_errors['Matrix element (a.u.)'] = matrixx
no_errors['Wavelength (nm)'] = wavell
no_errors['Transition Rate (s-1)'] = transrr
no_errors['Branching ratio'] = branchh
#no_errors

In [93]:
#sanity check that no branching ratio is greater than 1
for i,j in enumerate(no_errors['Branching ratio']):
    if float(j) > 1: 
        print(i, j, no_errors['Initial'][i], no_errors['Decay'][i], 'BRANCHING RATIO > 1!')
        break

In [94]:
# df = save_copy.copy()
# writer = pd.ExcelWriter('TransitionRates2.xlsx', engine='xlsxwriter') #2 because it won't have header
# df.to_excel(writer,sheet_name='welcome',index=False)
# writer.save()

In [95]:
# writer = pd.ExcelWriter('Lifetimes2.xlsx', engine='xlsxwriter') #2 because it won't have header
# Lifetimes.to_excel(writer,sheet_name='welcome',index=False)
# writer.save()

In [96]:
#SAVE FILE CSV
 
#save_copy.rename(columns = {'lifetimes': 'lifetimes (s)', 'wavelength': 'wavelength (Angstrom)'}, inplace = True)
#df.to_csv('TransitionRates2', sep= '\t', index = False)
#test = pd.read_csv('TransitionRates2', sep = ' ')
#all_state.to_pickle("TransitionLifeBranch.pkl")

In [97]:
Br_not_sci = [] #to get branching ratios out of scientific notation in html
Brer_not_sci = []
for i in range(len(excel_copy)):
    Br_not_sci.append(str(excel_copy['Branching Ratio'][i]))
    Brer_not_sci.append(str(excel_copy['Branching Ratio Error'][i]))
excel_copy['Branching Ratio'] = Br_not_sci
excel_copy['Branching Ratio Error'] = Brer_not_sci

In [98]:
# exp = pd.read_csv('Experimental_Data\Cs-matrix-elements.csv') #experiment
# a = list(save_copy.Initial)
# b = list(save_copy.Decay)
# c = list(zip(a,b))

# d = list(exp.From)
# e = list(exp.To)
# f = list(zip(d,e))

# #make list of indices, transitions rates, for me and Safronova. 
# comparison = []
# for i in range(len(c)):
#     try:
#         print(f.index(c[i]), exp['value'][f.index(c[i])], i)
#         testing.iloc[i, testing.columns.get_loc('matrix')] = exp['value'][f.index(c[i])]
#         testing.iloc[i, testing.columns.get_loc('mat_unc')] = exp['uncertainity'][f.index(c[i])]
#         testing.iloc[i, testing.columns.get_loc('mat_werr')] = exp['Matrixelement'][f.index(c[i])] + '*'
#         testing.iloc[i, testing.columns.get_loc('matrix')] = exp['value'][f.index(c[i])]
#         comparison.append((i, f.index(c[i]), save_copy['Transition Rate (s-1)'][i], exp['value'][f.index(c[i])], 
#                           exp['uncertainity'][f.index(c[i])]))
#     except ValueError:
#         #print(i)
#         #print(i, c[i], save_copy['mat_werr'][i])
#         pass

In [99]:
# exp = pd.read_csv('Experimental_Data\Cs-matrix-elements.csv') #experiment
# a = list(all_state.Initial)
# b = list(all_state.Decay)
# c = list(zip(a,b))

# d = list(exp.From)
# e = list(exp.To)
# f = list(zip(d,e))

# #make list of indices, transitions rates, for me and Safronova. 
# comparison = []
# for i in range(len(c)):
#     try:
#         comparison.append((i, f.index(c[i]), all_state['transition_rate (s-1)'][i], exp['value'][f.index(c[i])], 
#                           exp['uncertainty'][f.index(c[i])]))
#     except ValueError:
#         #print(i)
#         #print(i, c[i], save_copy['mat_werr'][i])
#         pass

In [100]:
html = save_copy.to_html(index = False)
html2 = Lifetimes.to_html(index = False)
html3 = no_errors.to_html(index = False)
html4 = excel_copy.to_html(index = False)
#print(html)

In [101]:
sub_scripts = ['1/2', '3/2', '5/2', '7/2', '9/2', '11/2']
htmls = [html, html2, html3, html4] #save_copy, Lifetimes, no_error, excel_copy
for i in sub_scripts:
    for j, k in enumerate(htmls):
        htmls[j] = htmls[j].replace('%s' % i, '<sub>%s</sub>' % i)

In [102]:
with open('save_copy.html', 'w') as fo:
    fo.write(htmls[0])

In [103]:
with open('Lifetimes.html', 'w') as fo:
    fo.write(htmls[1])

In [104]:
with open('no_errors.html', 'w') as fo:
    fo.write(htmls[2])

In [105]:
with open('excel_copy.html', 'w') as fo:
    fo.write(htmls[3])

In [113]:
from sigfig import round as oground

In [125]:
oground(78947, uncertainty = 1224, format = 'PDG', cutoff = 10)

'78900 ± 1000'

In [106]:
#pip uninstall beautifulsoup4

In [117]:
round(78947, uncertainty = 1224, format = 'Drake')

'7.89(12)E+04'

In [122]:
round(78947, uncertainty = 1224, format = 'Drake')

'7.89(12)E+04'

In [139]:
checkT

NameError: name 'checkT' is not defined

In [137]:
save_copy[317:318]

,Initial,Decay,Matrix element (a.u.),Wavelength (nm),Transition Rate (s-1),Branching ratio
317,9d3/2,10p3/2,31.446(84),133324(2),211.4(11),0.00004701(78)


In [126]:
round(78974, uncertainty = 1224)

79000

In [134]:
" ".join(re.findall("[0]", '79000')).replace(' ', '')

'000'

In [127]:
round(1224, 1)

1000

In [ ]:
#for wavelength, if number and error are >10 then
#round(wavelength, uncertainty = error) = #1
#round(uncertainty, 1) 1 is number of digits = #2
#value = #1.strip('0') + (#2) + " ".join(re.findall("[0]", #1)).replace(' ', '')

In [107]:
Break

NameError: name 'Break' is not defined

In [ ]:
#fix branching ratio errors in excel copy

In [ ]:
# df = save_copy.copy()

# # Create a Pandas Excel writer using XlsxWriter as the engine.
# writer = pd.ExcelWriter('pandas_simple.xlsx', engine='xlsxwriter')

# # Convert the dataframe to an XlsxWriter Excel object.
# df.to_excel(writer, sheet_name='Sheet1')

# # Get the xlsxwriter objects from the dataframe writer object.
# workbook  = writer.book
# worksheet = writer.sheets['Sheet1']

# format2 = workbook.add_format({'font_script': 2})
# worksheet.set_column('B:B', None, format2)
# workbook.close()

In [ ]:
# def get_digit(num):
#     if num < 10:
#         print(num)
#     else:
#         get_digit(num // 10)
#         print(num % 10)
# get_digit(1284.25)

###### def get_digit(number, n):
    return number // 10**n % 10
holder = []
number = 1280.569
for i in reversed(range(len(zeros))):
    holder.append(get_digit(number, i) - zeros[i])
holder

In [ ]:
# def get_digit(number, n):
#     return number // 10**n % 10
# def return_str(lst):
#     non_zero = lst.index(next(filter(lambda x: x!=0, lst)))
#     num_left = ''
#     for i in lst[non_zero:]:
#         num_left += str(i).split('.')[0]
#     return num_left

In [ ]:
#If I want to check if it matches rates1, rates2

# checkT = pd.read_csv(checkratesname, sep = '\s+', engine = 'python', header = None, skiprows = 1)
# col_names = ['Initial', 'Decay',     'Wavelength',     'UncE', 'UncE %',  'Matrix el.',    
#              'UncM',  'UncM %',    'Tran. rateE',     'Tran. rate',      'UncT',      'UncT %',   
#              'Br. ratio',     'UncB',    'UncB %'] 
# checkT.columns = col_names

# df1 = save_copy.copy()
# df2 = checkT.copy()

# a = list(save_copy.Initial)
# b = list(save_copy.Decay)
# c = list(zip(a,b))

# d = list(checkT.Initial)
# e = list(checkT.Decay)
# f = list(zip(d,e))

# #make list of indices, transitions rates, for me and Safronova. 
# comparison = []
# for i in range(len(c)):
#     try:
#         comparison.append((i, f.index(c[i]), save_copy['Transition Rate (s-1)'][i], checkT['Tran. rate'][f.index(c[i])], 
#                           checkT['Tran. rateE'][f.index(c[i])]))
#     except ValueError:
#         #print(i)
#         #print(i, c[i], save_copy['mat_werr'][i])
#         pass
 
# #print where my trans. rate and hers do not match
# for i in range(len(comparison)):
#     q = comparison[i][4].find('(') - 1
#     if str(comparison[i][2])[0:q] == str(comparison[i][3])[0:q]:
#         pass
#     else:
#         print(i,comparison[i][2],comparison[i][3], comparison[i][4], q)
   
# #find the differences between the transition rate, and the rate shown with () error
# round_diff = []
# for i in range(len(comparison)):
#     num1 = comparison[i][4]
#     num2 = num1.split('(')[1]
#     num3 = num2.split(')')[1]
#     num4 = num1.split('(')[0] + num3
#     num5 = float(num4)
#     round_diff.append((comparison[i][0], comparison[i][1],np.abs(comparison[i][3] - num5)) )
    
# for i in round_diff:
#     if i[2] > 9000:
#         print(i)
        
# wave_sorted = checkT.sort_values(by=['Wavelength'])
# tran_sorted = checkT.sort_values(by = ['Tran. rate'])

In [ ]:
# from decimal import Decimal
# from sortedcontainers import SortedDict
# from warnings import warn

# _manual_settings = {}
# _default_settings = {
#     'spacing': 0.1,
#     'spacer': '',
#     'decimal': '.',
#     'separator': ' ± ',
#     'cutoff': 9,
#     'prefix': False,
#     'exponent': 'E',
#     'round_by_sigfigs': False,
#     'round_by_decimals': True,
#     'given_sigfigs': 0}
# _major_prefixes = {
#     24: ['Y', 'septillion'],
#     21: ['Z', 'sextillion'],
#     18: ['E', 'quintillion'],
#     15: ['P', 'quadrillion'],
#     12: ['T', 'trillion'],
#     9: ['G', 'billion'],
#     6: ['M', 'million'],
#     3: ['k', 'thousand'],
#     0: ['',''],
#     -3: ['m', 'thousandth'],
#     -6: ['μ', 'millionth'],
#     -9: ['n', 'billionth'],
#     -12: ['p', 'trillionth'],
#     -15: ['f', 'quadrillionth'],
#     -18: ['a', 'quintillionth'],
#     -21: ['z', 'sextillionth'],
#     -24: ['y', 'septillionth']}
# _minor_prefixes = {
#     2: ['h', 'hundred'],
#     1: ['da', 'ten'],
#     -1: ['d', 'tenth'],
#     -2: ['c', 'hundredth']}

# class _Number:
#     '''
#     Private data structure for storing & manipulating numbers
#     Attributes:
#         .map: - dict for number storage
#               - maps 10's power (key) to numeric value (value)
#               - i.e. 3.14 => .map = {0:3, -1: 1, -2: 4}
#     Attributes/Mehtods for inspection (getting values):
#         .sign:     string of either '+' or '-', denoting sign of stored number
#         .positive: bool True/False depending on number's sign
#         .negative: bool True/False depending on number's sign
#         .has_uncertainty: bool True/False depending on whether there is an associated uncertainty with this number
#         .max_power(): returns integer corresponding to number's highest populated 10's power
#         .min_power(): returns integer corresponding to number's lowest populated 10's power
#         .decimate(dict, _Number, bool, bool): returns string of all digits in given format
#             specifying spacing and non-standard decimal point, optional sign
#             optionally embedded uncertainty, and optional leading/trailing zeros
#         .output(type): returns number in given type
#     Methods for manipulation (changing the value):
#         .set_sign(str): function used to change/set the number's sign by passing '-' or '+'
#                         so that .positive, .negative, .sign don't need manual updating
#         .increment_power_by(int): (de)increments all keys in .map by given value
#         .round_by_decimals(int):  performs rounding operation to the given 10's power
#         .prefixify(str):          converts to Scientific or Engineering notation with optional SI prefix
#     '''
#     def __init__(self):
#         self.set_sign('+')
#         self.prefix = ''
#         self.has_uncertainty = False
#         self.map = SortedDict()
#         self.zero = False
#     def set_sign(self, sign='+'):
#         '''sets the number's sign'''
#         if sign == '+':
#             self.negative = False
#             self.positive = True
#             self.sign = '+'
#         elif sign == '-':
#             self.negative = True
#             self.positive = False
#             self.sign = '-'
#         else:
#             warn('sign must be "+" or "-", assuming positive')
#             self.negative = False
#             self.positive = True
#             self.set_sign('+')
#     def max_power(self):
#         '''returns integer corresponding to number's highest populated 10's power'''
#         return max(self.map)
#     def min_power(self):
#         '''returns integer corresponding to number's lowest populated 10's power'''
#         return min(self.map)
#     def increment_power_by(self, n):
#         '''(de)increments all keys in .map'''
#         tmp = SortedDict()
#         for key in self.map:
#             tmp[key + n] = self.map[key]
#         self.map = tmp
#     def round_by_decimals(self, decimals):
#         '''performs rounding operation to the given 10's power'''
#         last_power = -decimals
#         tmp_map = SortedDict({self.max_power(): 0})
#         tmp_map = SortedDict({last_power: 0})
#         last_tmp_power = self.max_power()
#         for key in range(self.max_power(), min(-decimals - 1, -1), -1):
#             if key < last_power and key >= 0 or key not in self.map:
#                 float(0)
#             #if key < last_power and key < 0:
#             #    Decimal(3)
#             #    continue
#             else:
#                 last_tmp_power = key
#                 tmp_map[key] = self.map[key]
#         if last_power == self.max_power() + 1:
#             if self.map[self.max_power()] >= 5:
#                 tmp_map[last_power] = 1
#             else:
#                 self.zero = True
#                 tmp_map = SortedDict({last_power:0})
#                 self.set_sign('+')
#         elif last_power > self.max_power():
#             self.zero = True
#             tmp_map = SortedDict({last_power:0})
#             self.set_sign('+')
#         elif last_tmp_power - 1 in self.map and self.map[last_tmp_power - 1] >= 5:
#             tmp_map[last_tmp_power] += 1
#             tmp_power = last_tmp_power
#             while tmp_map[tmp_power] == 10:
#                 tmp_map[tmp_power] = 0
#                 tmp_power += 1
#                 if tmp_power not in tmp_map:
#                     tmp_map[tmp_power] = 1
#                 else:
#                     tmp_map[tmp_power] += 1
#         self.map = tmp_map
#         '''if self.map == SortedDict({0:0}):
#             self.zero = True
#             self.set_sign('+')
#             if self.has_uncertainty:
#                 if last_power < 0:
#                     for p in range(-1, last_power - 1, -1):
#                         self.map[p] = 0
#                 elif last_power > 0:
#                     self.map = SortedDict({last_power:0})'''
#     def decimate(self, format, unc=None, zeropadding=True, sign=True):
#         '''
#         returns string of all digits in given format {spacing, spacer, decimal},
#         with unc=_Number for embedded uncertainty, and optional leading/trailing zeros & sign
#         '''
#         top = self.max_power()
#         bot = self.min_power()
#         if zeropadding:
#             top = max(top, 0)
#             bot = min(bot, 0)
#         if self.zero and not unc and top > 0:
#             top = 0
#         output = []
#         if sign and self.negative:
#             output.append('-')
#         for p in range(top, bot - 1, -1):
#             try:
#                 output.append(str(self.map[p]))
#             except:
#                 output.append('0')
#             if p == self.min_power() and unc:
#                 output.append('('+unc.decimate(format, zeropadding=False, sign=False)+')')
#             if p != bot:
#                 if p == 0:
#                     output.append(format['decimal'])
#                 elif p % format['spacing'] == 0:
#                     output.append(format['spacer'])
#         return ''.join(output)
#     def output(self, output_type):
#         '''returns number in given type'''
#         no_formatting = {'decimal': '', 'spacer': '', 'spacing': 0.1}
#         return output_type(Decimal(self.decimate(no_formatting, zeropadding=False) +\
#                                     'E' + str(self.min_power())))
#     def __gt__(self, other):
#         if self.max_power() > other.max_power():
#             return True
#         for p in range(self.max_power(), self.min_power() - 1, -1):
#             if p not in other.map:
#                 if not self.map[p]:
#                     continue
#                 return True
#             if self.map[p] > other.map[p]:
#                 return True
#             if self.map[p] < other.map[p]:
#                 return False
#         if other.min_power() > self.min_power():
#             return False
#         return False
#     def prefixify(self, prefix, exponent):
#         '''converts to Engineering/Scientific notation with optional SI prefix'''
#         #self.prefix = 'XXX'
#         #return None
#         self.prefix = ''
#         power_shift = 0
#         if prefix == 'sci':
#             p = self.max_power()
#             self.prefix = exponent + str(p)
#             self.increment_power_by(-p)
#             power_shift += -p
#             return power_shift
#         elif prefix == 'eng':
#             p0 = 0
#             if self.max_power() < 0:
#                 p0 = 2
#             p = int((self.max_power() - p0)/3)*3
#             self.prefix = exponent + str(p)
#             self.increment_power_by(-p)
#             power_shift += -p
#             return power_shift
#         elif prefix in ['all', 'minor']:
#             prefixes = SortedDict(_major_prefixes+_minor_prefixes)
#             p = self.min_power()
#             while p >= min(prefixes):
#                 if p in prefixes:
#                     break
#                 p -= 1
#         else:
#             #prefix in [True, 'major', 'eng']:
#             prefixes = _major_prefixes
#             #p = int((self.min_power()-1)/3)*3
#             p = self.max_power()
#             while p > max(prefixes)+3:
#                 self.prefix += prefixes[max(prefixes)][0]
#                 self.increment_power_by(-max(prefixes))
#                 power_shift += -max(prefixes)
#                 p -= max(prefixes)
#             while p < min(prefixes):
#                 self.prefix += prefixes[min(prefixes)][0]
#                 self.increment_power_by(-min(prefixes))
#                 power_shift += -min(prefixes)
#                 p -= min(prefixes)
#             p0 = 0
#             if self.max_power() < 0:
#                 p0 = 2
#             p = int((self.max_power() - p0)/3)*3
#         self.prefix = prefixes[p][0] + self.prefix
#         self.increment_power_by(-p)
#         power_shift += -p
#         return power_shift

# def _arguments_parse(args, kwargs):
#     '''Private function for use only in round() function:
#     Deciphers user intent based on given inputs along with preset defaults
#     which returns actionable and sumarized useful variables in a dict.
#     '''
#     types = {int, str, Decimal, float, type(_Number()), type(None)}
#     given = {'output_type': type(args[0])}
#     if type(args[0]) not in types:
#         raise TypeError('Invalid input type of %s, expecting 1 of %s' % (type(args[0]), str(types)))
#     given['num'] = _num_parse(args[0])
#     if len(args) >= 2:
#         if type(args[1]) == int:
#             given['sigfigs'] = args[1]
#             if given['sigfigs'] < 1:
#                 warn('cannot have less that 1 significant figure, setting to 1')
#                 given['sigfigs'] = 1
#         else:
#             given['uncertainty'] = _num_parse(args[1])
#             given['output_type'] = str
#         #throw_exception("invalid 2nd argument, expecting number, got %s" % args[1])
#     if len(args) > 2:
#         warn("last %d argument(s) discarded/ignored" % int(len(args) - 3))
    
#     for key in _manual_settings:
#         given[key] = _manual_settings[key]

#     keys = {'separator', 'separation', 'sep', 'format', 'sigfigs', 'decimals', 'uncertainty', 'cutoff', 'spacing', 'spacer', 'decimal', 'output_type', 'output', 'type', 'style', 'prefix', 'notation', 'form', 'crop'}
#     for key in kwargs:
#         val = kwargs[key]
#         if key not in keys:
#             warn('unregonized argument, skipping %s=%s' % (key, val) )
#             continue
#         if key in given and key not in _manual_settings:
#             None
#             #warn("overwriting %s=%s with %s=%s" % (key, given[key], key, val))
#         if key in {'sigfigs', 'decimals', 'cutoff', 'crop', 'spacing'}:
#             try:
#                 if key == 'crop':
#                     key = 'cutoff'
#                 if type(val) in [float, Decimal]:
#                     warn('use integer type for %s argument' % key)
#                 if key in {'cutoff', 'crop'} and int(val) < 9:
#                     warn('cutoff/crop cannot be < 9, setting to 9')
#                     val = 9
#                 if key == 'sigfigs' and int(val) < 1:
#                     warn('cannot have less that 1 significant figure, setting to 1')
#                     val = 1
#                 given[key] = int(val)
#             except:
#                 warn('Ignoring %s=%s, invalid type of %s, expecting integer type' % (key, val, type(val)))
#         elif key == 'uncertainty':
#             given['uncertainty'] = _num_parse(val)
#         elif key == 'prefix':
#             if type(val) == bool or val in {'major', 'sci', 'eng'}:
#                 given[key] = val
#             elif val in ['minor', 'all']:
#                 given['prefix'] = 'all'
#             else:
#                 warn('Ignoring %s=%s, invalid prefix setting, expecting: %s' % (key, val, str([True, False, 'major', 'minor', 'all'])))
#                 continue
#             given['output_type'] = str
#         elif key in {'spacer', 'decimal'}:
#             given[key] = str(val)
#         elif key in {'sep', 'separation', 'separator'}:
#             if val == 'external_brackets':
#                 given['separator'] = 'brackets'
#                 given['external_brackets'] = True
#                 continue
#             elif val == 'brackets':
#                 given['separator'] = 'brackets'
#             elif val in {tuple, list}:
#                 given['output'] = val
#                 given['output_type'] = type(args[0])
#             elif val == 'tuple':
#                 given['output'] = tuple
#                 given['output_type'] = type(args[0])
#             elif val == 'list':
#                 given['output'] = list
#                 given['output_type'] = type(args[0])
#             given['separator'] = str(val)
#         elif key in ['format', 'style', 'output', 'type', 'output_type', 'notation', 'form']:
#             given['output_type'] = str
#             #warning might be warranted if output_type previously specified
#             properties = ['spacing', 'spacer', 'decimal', 'separator', 'cutoff', 'prefix', 'form']
#             formats = {'English': [3, ',', '.', ' ± ', 9, False,  '#,###,###.## ± 0.#'],
#                        'French':  [3, ' ', ',', ' ± ', 99, False, '# ### ###,## ± 0,##'],
#                        'other':   [3, '.', ',', ' ± ', 99, False, '#.###.###,## ± 0,##'],
#                        'PDG':     [.1, '', '.', ' ± ', 35, False,  '# ### ###.##(##)'],
#                        'Drake':   [3, ' ', '.', 'brackets', 29, False, '# ### ###.##(##)'],
#                        'sci':     [.1, '', '.', ' ± ', 9, 'sci', '# ### ###.##(##)'],
#                        'eng':     [.1, '', '.', ' ± ', 9, 'eng', '# ### ###.##(##)'],
#                        'std':     [.1, '', '.', ' ± ', 9, False, '# ### ###.##(##)']}
#             outputs = {'+-', 'map'}
#             notations = {'sci', 'scientific', 'eng', 'engineering', 'std', 'standard'}
#             if val in notations:
#                 val = val[:3]
#                 if val == 'sta':
#                     val = 'std'
#                 for i, prop in enumerate(properties):
#                     #if prop in given and key not in _manual_settings:
#                     if prop in given:
#                         continue
#                         #warn("overwriting %s=%s with %s=%s" % (prop, given[prop], prop, formats[val][i]))
#                     given[prop] = formats[val][i]
#             elif val in formats:
#                 for i, prop in enumerate(properties):
#                     if prop in given and key not in _manual_settings:
#                         None
#                         #warn("overwriting %s=%s with %s=%s" % (prop, given[prop], prop, formats[val][i]))
#                     given[prop] = formats[val][i]
#             elif val in types:
#                 given['output_type'] = val
#                 if 'prefix' in given:
#                     del given['prefix']
#             elif val in outputs:
#                 if val == '+-':
#                     given['separator'] = ' ± '
#                 else:
#                     given['output'] = val
#             else:
#                 warn("expected format of %s, ignoring format of %s" % ([f for f in formats] + [o for o in outputs], val))
#                 given['output_type'] = type(args[0])
#         else:
#             given[key] = val

#     if 'sigfigs' in given and 'decimals' in given:
#         warn('Cannot round by both sigfigs & decimals, ignoring decimal constraint')
#         del given['decimals']
#     if 'uncertainty' in given and any([x in given for x in ['sigfigs', 'decimals', 'arg2']]):
#         warn('Cannot round by both uncertainty & decimals/sigfigs simultaneously, ignoring decimals &/or sigfigs.  Use seperate calls to round() function for seperate roundings.')
#         for prop in ['sigfigs', 'decimals', 'arg2']:
#             if prop in given:
#                 del given[prop]
#     if 'arg2' in given:
#         if 'sigfigs' or 'decimals' in given:
#             warn('Invalid 2nd argument, ignoring.  Sigfigs or decimals given in keyword arguments')
#         elif _default_settings['round_by_sigfigs']:
#             given['sigfigs'] = given['arg2']
#         else:
#             given['decimals'] = given['arg2']
#         del given['arg2']
#     if given['output_type'] not in {float, Decimal, type(_Number())}:
#         given['format'] = {}
#         for prop in {'decimal', 'spacer', 'spacing'}:
#             if prop in given:
#                 val = given[prop]
#                 del given[prop]
#             elif prop in _manual_settings:
#                 val = _manual_settings[prop]
#             else:
#                 val = _default_settings[prop]
#             given['format'][prop] = val
#     for prop in ['separator', 'prefix', 'exponent']:
#         if prop in given:
#             continue
#         elif prop in _manual_settings:
#             given[prop] = _manual_settings[prop]
#         else:
#             given[prop] = _default_settings[prop]
     
#     return given
# def _num_parse(num):
#     '''Private function for use only in round()'s _arguments_parse() function:
#     Translates given number of any type into returned _Number data structure
#     Parsing Algorythm [O(N)]:
#     - convert to string
#     - characters are analyzed sequentially in a KMP-like state graph. ie:
#        number: -325.7854E-5 
#        state:  ABBBBCCCCDEE
#     - ValueError is raised if input number cannot be deciphered
#     '''
#     global number, i, n, negative_exp, exp
#     number = _Number()
#     i = 0
#     n = 0
#     negative_exp = False
#     exp = 0

#     if type(num) == type(number):
#         return num
#     if not num:
#         warn('no number provided, assuming zero (0)')
#         number = _Number()
#         number.map[0] = 0
#         number.zero = True
#         return number
#     num = str(num)

#     digits = set([str(a) for a in range(10)])
#     exponents = set(['E', 'e', 'D', 'd', 'Q', 'q'])

#     def A(num):
#         global number, i, n
#         i += 1
#         if not num or num in '.-+':
#             warn('no number provided, assuming zero (0)')
#             number.map[0] = 0
#             number.zero = True
#             return None
#         elif num[0] in exponents:
#             warn('no number provided, assuming zero (0)')
#             number.map[0] = 0
#             number.zero = True
#             D(num[1:])
#         elif num[0] in '+-':
#             number.set_sign(num[0])
#             B(num[1:])
#         elif num[0] in '.':
#             C(num[1:])
#         elif num[0] in digits:
#             n += 1
#             number.map[-n] = int(num[0])
#             B(num[1:])
#         else:
#             raise ValueError('invalid input Character "%s" (position %d, state A)' % (num[0], i))
#     def B(num):
#         global number, i, n
#         i += 1
#         if not num:
#             number.increment_power_by(n)
#             return None
#         elif num[0] in exponents:
#             number.increment_power_by(n)
#             D(num[1:])
#         elif num[0] in '.':
#             number.increment_power_by(n)
#             n = 0
#             C(num[1:])
#         elif num[0] in digits:
#             n += 1
#             number.map[-n] = int(num[0])
#             B(num[1:])
#         else:
#             raise ValueError('invalid Character "%s" (position %d, state B)' % (num[0], i))
#     def C(num):
#         global number, i, n
#         i += 1
#         if not num:
#             return None
#         elif num[0] in exponents:
#             D(num[1:])
#         elif num[0] in digits:
#             n += 1
#             number.map[-n] = int(num[0])
#             C(num[1:])
#         else:
#             raise ValueError('invalid Character "%s" (position %d, state C)' % (num[0], i))
#     def D(num):
#         global i, negative_exp, exp
        
#         i += 1
#         if not num:
#             warn('exponent expected but not provided')
#             return None
#         elif num[0] in '+-':
#             if num[0] == '-':
#                 negative_exp = True
#             if not num[1:]:
#                 warn('exponent expected but not provided')
#             exp = 0
#             E(num[1:])
#         elif num[0] in digits:
#             exp = int(num[0])
#             E(num[1:])
#         else:
#             raise ValueError('invalid Character "%s" (position %d, state D)' % (num[0], i))
#     def E(num):
#         global number, i, exp
#         i += 1
#         if not num:
#             sign = 1
#             if negative_exp:
#                 sign = -1
#             number.increment_power_by(sign*exp)
#             return None
#         elif num[0] in digits:
#             exp = 10*exp + int(num[0])
#             E(num[1:])
#         else:
#             raise ValueError('invalid Character "%s" (position %d, state E)' % (num[0], i))

#     A(num)

#     p = number.max_power()
#     while p in number.map and number.map[p] == 0:
#         del number.map[p]
#         p -= 1

#     if not number.map:
#         number.map[0] = 0
#         number.zero = True

#     return number

# def round(*args, **kwargs):
#     '''
#     round a given number to either
#         - a given number of significant figures/significant digits,
#         - a given number of decimals, or
#         - a given uncertainty
#     and optionally output the number with given formatting along with its uncertainty
#     Key usage examples:
#         round('2.675', sigfigs=2) => '2.7'
#         round(2.675, decimals=2)  =>  2.68
#         round('2.675', '0.00197') => '2.675 ± 0.002'
#         round('123456.789099', '-1.15E-4', output='Drake') => '123 456.789 10(12)
#     For detailed usage instructions see https://pypi.org/project/sigfig/
#     '''
#     if not args:
#         warn("no input number given, nothing to return")
#         return None
#     given = _arguments_parse(args, kwargs)
#     print(args)
#     print(kwargs)
#     print(given.keys())
#     print(given['prefix'])
#     num = given['num']
    
#     if 'decimals' in given:
#         num.round_by_decimals(given['decimals'])
#     elif 'sigfigs' in given:
#         if given['sigfigs'] > len(num.map):
#             warn("warning: %d significant figures requested from number with only %d significant figures" % (given['sigfigs'], len(num.map)))
#         last_power = num.max_power() - given['sigfigs'] + 1
#         num.round_by_decimals(-last_power)
#         while len(num.map) > given['sigfigs']:
#             del num.map[num.min_power()]
#     elif 'uncertainty' in given:
#         num.has_uncertainty = True
#         if 'cutoff' in given:
#             #here could be the wavelength check?
#             cutoff = str(given['cutoff'])
#         elif 'cutoff' in _manual_settings:
#             cutoff = str(_manual_settings['cutoff'])
#         else:
#             cutoff = str(_default_settings['cutoff'])
#         unc = round(given['uncertainty'], sigfigs=len(cutoff), output='map')
#         cut = _num_parse(cutoff + 'E' + str(unc.min_power()))
# #         if unc > cut:
# #             unc = round(given['uncertainty'], sigfigs=len(cutoff)-1, output='map')
# #             if unc.map[unc.max_power()] == 1:
# #                 unc.map[unc.max_power() - 1] = 0
#         num.round_by_decimals(-unc.min_power())
    
#     if given['prefix']:
#         power_shift = num.prefixify(given['prefix'], given['exponent'])
#         if 'uncertainty' in given:
#             unc.increment_power_by(power_shift)

#     if given['output_type'] in [int, Decimal, float]:
#         if 'output' in given and given['output'] in {tuple, list}:
#             if 'uncertainty' not in given:
#                 return given['output']([num.output(given['output_type'])])
#             return given['output']([num.output(given['output_type']),
#                                     unc.output(given['output_type'])])
#         return num.output(given['output_type'])
#     elif 'output' in given and given['output'] == 'map':
#         return num
#     if 'uncertainty' in given and given['separator'] == 'brackets' and unc.min_power() > 0 and 'external_brackets' not in given:
#         if len(str(num.decimate(given['format'], unc=unc)).split(')')[1]) > 0: #there are numbers after the () format of error, Drake does this when error is great than 1
#             er_num = re.search('\(([^)]+)', str(num.decimate(given['format'], unc=unc))).group(1).replace(' ', '')
#             precise = len(str(num.decimate(given['format'], unc = unc)).split('(')[0].replace(' ', '')) - len(er_num) + 1
#             g_num = np.format_float_scientific(float(str(num.decimate(given['format'])).replace(' ', '')), 
#                                                unique = False, precision = precise)
#             final_num = g_num.split('e')[0] + '(%s)' % er_num + 'E' + g_num.split('e')[1]
#             return(final_num)
#         return str(num.decimate(given['format'], unc=unc)).replace(' ', '')

#     output = num.decimate(given['format'])
#     if 'output' in given and given['output'] in {list, tuple}:
#         if 'uncertainty' in given:
#             return given['output']([output, unc.decimate(given['format'], sign=False)])
#         return given['output']([output])
#     if 'uncertainty' in given:
#         if given['separator'] == 'brackets' and unc.min_power() > 0:
#             output += '('+unc.decimate(given['format'], sign=False)+')'
#         elif given['separator'] == 'brackets':
#             error_formatted = str('('+unc.decimate(given['format'], zeropadding=False, sign=False)+')')
#             error_formatted = error_formatted.replace('.', '')
#             if float(error_formatted.replace('(', '').replace(')', '').replace(' ', '')) != 0:
#                 output += error_formatted
#             else:
#                 output = output
#             #output += '('+unc.decimate(given['format'], zeropadding=False, sign=False)+')'
#         else:
#             if given['prefix'] and given['prefix'] not in {True, 'major', 'minor', 'all'}:
#                 output += num.prefix
#             output += given['separator'] + unc.decimate(given['format'], sign=False)

#     if given['prefix']:
#         output += num.prefix

#     return output.replace(' ', '')

# def roundit(*args, **kwargs):
#     '''Depreciated version of round() function with limited scope'''
#     warn('Depreciated Usage: Migrate code to use round() function instead', DeprecationWarning)
#     defaults = {'spacer': ' ', 'spacing': 3, 'separator': 'brackets', 'output_type' : str}
#     final_parameters = defaults
#     if 'form' in kwargs:
#         if kwargs['form'] == 'plusminus':
#             kwargs['separator'] = ' +/- '
#         else:
#             kwargs['separator'] = kwargs['form']
#         del kwargs['form']
#     if 'crop' in kwargs:
#         kwargs['crop'] -= 1
#     for key in kwargs:
#         final_parameters[key] = kwargs[key]
#     return round(*args, **final_parameters)
# def round_unc(*args, **kwargs):
#     '''Depreciated version of round() function with limited scope'''
#     warn('Depreciated Usage: Migrate code to use round() function instead', DeprecationWarning)
#     defaults = {'sep': tuple}
#     final_parameters = defaults
#     if 'form' in kwargs and kwargs['form'] == 'plusminus':
#         kwargs['separator'] = ' +/- '
#         del kwargs['form']
#     if 'crop' in kwargs:
#         kwargs['crop'] -= 1
#     for key in kwargs:
#         final_parameters[key] = kwargs[key]
#     return round(*[str(arg) for arg in args], **final_parameters)[0]
# def round_sf(number, sigfigs):
#     '''Depreciated version of round() function with limited scope'''
#     warn('Depreciated Usage: Migrate code to use round() function instead', DeprecationWarning)
#     return round(str(number),sigfigs=sigfigs)
# round('99189.93706786614', '1810.7405415399924', format = 'Drake')